# Exploring The Dimensions Search Language (DSL) - Deep Dive

This tutorial provides a detailed walkthrough of the most important features of the [Dimensions Search Language](https://docs.dimensions.ai/dsl/). 

This tutorial is based on the [Query Syntax](https://docs.dimensions.ai/dsl/language.html) section of the official documentation. So, it can be used as an interactive version of the documentation, as it allows to try out the various DSL queries presented there.

## What is the Dimensions Search Language?

The DSL aims to capture the type of interaction with Dimensions data
that users are accustomed to performing graphically via the [web
application](https://app.dimensions.ai/), and enable web app developers, power users, and others to
carry out such interactions by writing query statements in a syntax
loosely inspired by SQL but particularly suited to our specific domain
and data organization.

**Note:** this notebook uses the Python programming language, however all the **DSL queries are not Python-specific** and can in fact be reused with any other API client. 



## Prerequisites

This notebook assumes you have installed the [Dimcli](https://pypi.org/project/dimcli/) library and are familiar with the *Getting Started* tutorial.


In [1]:
!pip install dimcli --quiet 

import dimcli
from dimcli.utils import *
import json
import sys
import pandas as pd
#

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
  import getpass
  KEY = getpass.getpass(prompt='API Key: ')  
  dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
  KEY = ""
  dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v0.8.2.2)
Connected to: https://app.dimensions.ai - DSL v1.28
Method: dsl.ini file



## Sections Index 

1. Basic query structure
2. Full-text searching
3. Field searching
4. Searching for researchers
5. Returning results 
6. Aggregations

## 1. Basic query structure

DSL queries consist of two required components: a `search` phrase that
indicates the scientific records to be searched, and one or
more `return` phrases which specify the contents and structure of the
desired results.

The simplest valid DSL query is of the form `search <source>|return <result>`:

In [2]:
%%dsldf 
search grants return  grants limit 5

Returned Grants: 5 (total = 5623964)
Time: 0.80s


project_num    end_date                           active_year  \
0          890218  2023-11-30                    [2021, 2022, 2023]   
1  2018-HRSI-1548         NaN                                [2021]   
2          948141  2026-10-31  [2021, 2022, 2023, 2024, 2025, 2026]   
3          887019  2023-10-31                    [2021, 2022, 2023]   
4    AH/V001841/1  2023-04-02                    [2021, 2022, 2023]   

                                      original_title  start_year  \
0  Functional analysis of ribosome heterogeneity ...        2021   
1  APPROACH to Enriching the Real World Evidence ...        2021   
2  Simulating ultracold correlated quantum matter...        2021   
3  The role of microbial Oxylipins in the MIcrobe...        2021   
4  Playgrounds: A Material Cultural Study of Post...        2021   

              id                          funding_org_name title_language  \
0  grant.9064785                       European Commission             en   
1  grant.8690978  New Brunswick Health Research Foundation             en   
2  grant.9414093                 European Research Council             en   
3  grant.8964187                       European Commission             en   
4  grant.9401634      Arts and Humanities Research Council             en   

  language                                              title  start_date  \
0       en  Functional analysis of ribosome heterogeneity ...  2021-12-01   
1       en  APPROACH to Enriching the Real World Evidence ...  2021-11-30   
2       en  Simulating ultracold correlated quantum matter...  2021-11-01   
3       en  The role of microbial Oxylipins in the MIcrobe...  2021-11-01   
4       en  Playgrounds: A Material Cultural Study of Post...  2021-10-03   

                                             funders  
0  [{'id': 'grid.270680.b', 'country_name': 'Belg...  
1  [{'id': 'grid.484521.e', 'country_name': 'Cana...  
2  [{'id': 'grid.452896.4', 'country_name': 'Belg...  
3  [{'id': 'grid.270680.b', 'country_name': 'Belg...  
4  [{'id': 'grid.426413.6', 'country_name': 'Unit...

### `search source`

A query must begin with the word `search` followed by a `source` name, i.e. the name of a type of scientific `record`, such as `grants` or `publications`.

**What are the sources available?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [3]:
dsl.query("describe schema")

<dimcli.DslDataset object #4479630496. Dict keys: 'sources', 'entities'>

A more useful query might also make use of the optional `for` and
`where` phrases to limit the set of records returned.

In [4]:
%%dsldf 
search grants  for "lung cancer" 
    where active_year=2000 
return  grants  limit 5

Returned Grants: 5 (total = 1734)
Time: 0.59s


title_language                                            funders  \
0             en  [{'id': 'grid.279885.9', 'acronym': 'NHLBI', '...   
1             en  [{'id': 'grid.279885.9', 'acronym': 'NHLBI', '...   
2             en  [{'id': 'grid.279885.9', 'acronym': 'NHLBI', '...   
3             en  [{'id': 'grid.279885.9', 'acronym': 'NHLBI', '...   
4             en  [{'id': 'grid.419213.c', 'acronym': 'RWJF', 'c...   

                                      original_title  \
0   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE   
1  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...   
2  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...   
3  Synthetic Heparan Sulfate: Probing Biosynthesi...   
4          SmokeLess States Program - Implementation   

                          funding_org_name  project_num             id  \
0  National Heart Lung and Blood Institute  F32HL010455  grant.2386513   
1  National Heart Lung and Blood Institute  R01HL063695  grant.2537116   
2  National Heart Lung and Blood Institute  R01HL066221  grant.2537801   
3  National Heart Lung and Blood Institute  R01HL062244  grant.2536777   
4           Robert Wood Johnson Foundation        41067  grant.8616620   

                                               title  start_date    end_date  \
0   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE  2000-12-31  2002-01-01   
1  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...  2000-12-18  2004-11-30   
2  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...  2000-12-18  2007-11-30   
3  Synthetic Heparan Sulfate: Probing Biosynthesi...  2000-12-15  2017-12-31   
4          SmokeLess States Program - Implementation  2000-12-01  2001-02-28   

  language                                        active_year  start_year  
0       en                                 [2000, 2001, 2002]        2000  
1       en                     [2000, 2001, 2002, 2003, 2004]        2000  
2       en   [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]        2000  
3       en  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...        2000  
4       en                                       [2000, 2001]        2000

### `return` result (source or facet)

The most basic `return` phrase consists of the keyword `return` followed
by the name of a `record` or `facet` to be returned. 

This must be the
name of the `source` used in the `search` phrase, or the name of a
`facet` of that source.

In [5]:
%%dsldf
search grants for "laryngectomy" 
return grants limit 5

Returned Grants: 5 (total = 117)
Time: 0.54s


title_language                                            funders  \
0             ja  [{'id': 'grid.54432.34', 'acronym': 'JSPS', 'c...   
1             ja  [{'id': 'grid.54432.34', 'acronym': 'JSPS', 'c...   
2             en  [{'id': 'grid.453041.7', 'acronym': 'DCS', 'co...   
3             en  [{'id': 'grid.421091.f', 'acronym': 'EPSRC', '...   
4             en  [{'id': 'grid.214431.1', 'acronym': 'NIDCD', '...   

                                      original_title  \
0            喉頭全摘出者の家族の術後生活への移行を促進する外来での生活支援プログラムの開発   
1                   表情解析ソフトウェアを用いた自己音声再建による新規代用音声の開発   
2  Prophylactic pectoralis major flap to compensa...   
3  UKRI CDT in SLT- Continuous End-to-End Streami...   
4  Wearable silent speech technology to enhance i...   

                                    funding_org_name  project_num  \
0         Japan Society for the Promotion of Science     20K10777   
1         Japan Society for the Promotion of Science     20K18439   
2                               Dutch Cancer Society        12483   
3  Engineering and Physical Sciences Research Cou...      2268211   
4  National Institute on Deafness and Other Commu...  R01DC016621   

              id                                              title  \
0  grant.9201764  Development of an outpatient life support prog...   
1  grant.9209426  Development of new substitute voice by self-vo...   
2  grant.9387466  Prophylactic pectoralis major flap to compensa...   
3  grant.8674095  UKRI CDT in SLT- Continuous End-to-End Streami...   
4  grant.8554260  Wearable silent speech technology to enhance i...   

   start_date    end_date language                           active_year  \
0  2020-04-01  2024-03-31       ja        [2020, 2021, 2022, 2023, 2024]   
1  2020-04-01  2022-03-31       ja                    [2020, 2021, 2022]   
2  2020-02-01  2024-02-01       nl        [2020, 2021, 2022, 2023, 2024]   
3  2019-09-29  2023-09-28       en        [2019, 2020, 2021, 2022, 2023]   
4  2019-08-15  2024-07-31       en  [2019, 2020, 2021, 2022, 2023, 2024]   

   start_year  
0        2020  
1        2020  
2        2020  
3        2019  
4        2019

Eg let's see what are the *facets* available for the *grants* source:

In [6]:
fields = dsl.query("describe schema")['sources']['grants']['fields']
[x for x in fields if fields[x]['is_facet']]

['funder_countries',
 'research_org_state_codes',
 'funding_currency',
 'funding_org_city',
 'funders',
 'active_year',
 'language',
 'research_org_countries',
 'research_org_cities',
 'category_sdg',
 'category_for',
 'category_uoa',
 'funding_org_name',
 'research_orgs',
 'start_year',
 'category_icrp_cso',
 'category_hrcs_hc',
 'language_title',
 'category_bra',
 'researchers',
 'category_hra',
 'funding_org_acronym',
 'category_hrcs_rac',
 'category_icrp_ct',
 'category_rcdc']

## 2. Full-text Searching

Full-text search or keyword search finds all instances of a term
(keyword) in a document, or group of documents. 

Full text search works
by using search indexes, which can be targeting specific sections of a
document e.g. its $abstract$, $authors$, $full text$ etc...

In [7]:
%%dsldf 
search publications 
    in full_data for "moon landing" 
return publications limit 5

Returned Publications: 5 (total = 184216)
Time: 1.29s


author_affiliations    pages              id  \
0  [[{'raw_affiliation': ['Centro de Investigacio...  105-123  pub.1134005335   
1  [[{'raw_affiliation': ['Institute of Chemistry...   101596  pub.1132933847   
2  [[{'raw_affiliation': ['Departamento de Matemá...  424-437  pub.1133496951   
3  [[{'raw_affiliation': ['School of Chemical Eng...   118238  pub.1133999332   
4  [[{'raw_affiliation': ['German Aerospace Cente...    87-97  pub.1133275673   

   year     type                                              title volume  \
0  2021  article  Participatory Mapping and PGIS: Secerning Fact...     10   
1  2021  article  Critical review of chirality indicators of ext...     92   
2  2021  article  Modeling of Enhanced Micro-Energy Harvesting o...    168   
3  2021  article  Continuous Microfluidic Solvent Extraction of ...    260   
4  2021  article  Green bipropellant development – A study on th...    226   

  issue    journal.id                                 journal.title  
0     3  jour.1147177  International Journal of E-Planning Research  
1   NaN  jour.1138083                         New Astronomy Reviews  
2   NaN  jour.1048491                              Renewable Energy  
3   NaN  jour.1380465        Separation and Purification Technology  
4   NaN  jour.1135200                          Combustion and Flame

### 2.1 `in [search index]`

This optional phrase consists of the particle `in` followed by a term indicating a `search index`, specifying for example whether the search
is limited to full text, title and abstract only, or title only. 

In [8]:
%%dsldf 
search grants 
    in title_abstract_only for "something" 
return grants limit 5

Returned Grants: 5 (total = 10290)
Time: 0.55s


project_num    end_date                           active_year  \
0        890630  2023-08-31                    [2021, 2022, 2023]   
1        949572  2026-01-31  [2021, 2022, 2023, 2024, 2025, 2026]   
2  NE/V005847/1  2025-01-03        [2021, 2022, 2023, 2024, 2025]   
3        865624  2025-12-31        [2021, 2022, 2023, 2024, 2025]   
4        895379  2021-12-31                                [2021]   

                                      original_title  start_year  \
0  Deciphering fundamental constraints on pathoge...        2021   
1  Statistical Host Identification As a Test of D...        2021   
2  Sustainable Plastic Attitudes to benefit Commu...        2021   
3  Overcoming stellar activity in radial velocity...        2021   
4  Cortical-to-Subcortical Information Transfer U...        2021   

              id                      funding_org_name title_language  \
0  grant.9064570                   European Commission             en   
1  grant.9385331             European Research Council             en   
2  grant.9414560  Natural Environment Research Council             en   
3  grant.8964099             European Research Council             en   
4  grant.9065191                   European Commission             en   

  language                                              title  start_date  \
0       en  Deciphering fundamental constraints on pathoge...  2021-09-01   
1       en  Statistical Host Identification As a Test of D...  2021-02-01   
2       en  Sustainable Plastic Attitudes to benefit Commu...  2021-01-04   
3       en  Overcoming stellar activity in radial velocity...  2021-01-01   
4       en  Cortical-to-Subcortical Information Transfer U...  2021-01-01   

                                             funders  
0  [{'id': 'grid.270680.b', 'country_name': 'Belg...  
1  [{'id': 'grid.452896.4', 'country_name': 'Belg...  
2  [{'id': 'grid.8682.4', 'country_name': 'United...  
3  [{'id': 'grid.452896.4', 'country_name': 'Belg...  
4  [{'id': 'grid.270680.b', 'country_name': 'Belg...

Eg let's see what are the *search fields* available for the *grants* source:

In [9]:
dsl.query("describe schema")['sources']['grants']['search_fields']

['concepts', 'investigators', 'full_data', 'title_only', 'title_abstract_only']

In [10]:
%%dsldf 
search grants 
    in full_data for "graphene AND computer AND iron" 
return grants limit 5

Returned Grants: 5 (total = 11)
Time: 0.56s


project_num    end_date         active_year  \
0          19-43-04129  2021-12-31  [2019, 2020, 2021]   
1          18-02-20097  2018-12-31              [2018]   
2    4491/E-370/S/2016  2016-12-31              [2016]   
3  2015/17/B/ST8/01422  2018-01-25  [2016, 2017, 2018]   
4    4491/E-370/S/2015  2015-12-31              [2015]   

                                      original_title  start_year  \
0  Weyl and Dirac semimetals and beyond - predict...        2019   
1  Проект организации 18-ой Международной конфере...        2018   
2  Dotacja podmiotowa na utrzymanie potencjału ba...        2016   
3  Katalizatory heterogeniczne na bazie nanostruk...        2016   
4  Dotacja podmiotowa na utrzymanie potencjału ba...        2015   

              id                          funding_org_name title_language  \
0  grant.8413990                Russian Science Foundation             en   
1  grant.8731867     Russian Foundation for Basic Research             ru   
2  grant.7397800  Ministry of Science and Higher Education             pl   
3  grant.7401748                   National Science Center             pl   
4  grant.7397795  Ministry of Science and Higher Education             pl   

  language                                              title  start_date  \
0       en  Weyl and Dirac semimetals and beyond - predict...  2019-01-01   
1       ru  Project of the organization of the 18th Intern...  2018-01-01   
2       pl  Subject subsidy for maintaining the research p...  2016-02-22   
3       pl  Heterogeneous catalysts based on carbon nanost...  2016-01-26   
4       pl  Subject subsidy for maintaining the research p...  2015-02-19   

                                             funders  
0  [{'id': 'grid.454869.2', 'country_name': 'Russ...  
1  [{'id': 'grid.452899.b', 'country_name': 'Russ...  
2  [{'id': 'grid.425823.a', 'country_name': 'Pola...  
3  [{'id': 'grid.436846.b', 'country_name': 'Pola...  
4  [{'id': 'grid.425823.a', 'country_name': 'Pola...

Special search indexes for persons names permit to perform full text
searches on publications `authors` or grants `investigators`. Please see the
*Researchers Search* section below for more information
on how searches work in this case.

In [11]:
%dsldf search publications in authors for "\"Jennifer A Doudna\"" return publications limit 5

Returned Publications: 5 (total = 344)
Time: 1.04s


id                                              title  year  \
0  pub.1133635406          Rapid detection of SARS-CoV-2 with Cas13.  2020   
1  pub.1133534923           Controlling and enhancing CRISPR systems  2020   
2  pub.1133449309  IGI-LuNER: single-well multiplexed RT-qPCR tes...  2020   
3  pub.1133370119  Corrigendum to “Engineering of monosized lipid...  2020   
4  pub.1133106825  Amplification-free detection of SARS-CoV-2 wit...  2020   

                                 author_affiliations      type    journal.id  \
0  [[{'raw_affiliation': [], 'first_name': 'Shree...  preprint  jour.1369542   
1  [[{'raw_affiliation': ['Department of Molecula...   article  jour.1327431   
2  [[{'raw_affiliation': [], 'first_name': 'Eliza...  preprint  jour.1369542   
3  [[{'raw_affiliation': ['Department of Chemical...   article  jour.1034525   
4  [[{'raw_affiliation': ['J. David Gladstone Ins...   article  jour.1019114   

             journal.title volume issue  pages  
0                  medRxiv    NaN   NaN    NaN  
1  Nature Chemical Biology     17     1  10-19  
2                  medRxiv    NaN   NaN    NaN  
3       Acta Biomaterialia    NaN   NaN    NaN  
4                     Cell    NaN   NaN    NaN

### 2.2 `for "search term"`

This optional phrase consists of the keyword `for` followed by a
`search term` `string`, enclosed in double quotes (`"`).

Strings in double quotes can contain nested quotes escaped by a
backslash `\`. This will ensure that the string in nested double quotes
is searched for as if it was a single phrase, not multiple words.

An example of a phrase: `"\"Machine Learning\""` : results must contain
`Machine Learning` as a phrase.

In [12]:
%dsldf search publications for "\"Machine Learning\"" return publications limit 5

Returned Publications: 5 (total = 1332770)
Time: 1.01s


title issue     type  \
0  K-nearest neighbor and naïve Bayes based diagn...     6  article   
1  Evaluation of Intersection Properties Using MA...     6  article   
2  Priority-based low voltage DC microgrid system...   NaN  article   
3  A Closed-Form Solution to Estimate Spatially V...    99  article   
4  Design and Implementation of a Wind Farm Contr...    99  article   

               id                                author_affiliations volume  \
0  pub.1130334381  [[{'raw_affiliation': ['Universiti Teknikal Ma...      9   
1  pub.1129037745  [[{'raw_affiliation': [], 'first_name': 'Görke...     32   
2  pub.1132780778  [[{'raw_affiliation': ['Department of Electric...      7   
3  pub.1133333464  [[{'raw_affiliation': ['DIFFER-Dutch Institute...     PP   
4  pub.1133394955  [[{'raw_affiliation': ['Knowledge Exchange Fel...     PP   

       pages  year    journal.id  \
0  2650-2657  2021  jour.1144063   
1        NaN  2021  jour.1226975   
2      43-51  2021  jour.1150945   
3        1-1  2021  jour.1293397   
4        1-1  2021  jour.1293397   

                                       journal.title  
0  Bulletin of Electrical Engineering and Informa...  
1                                       Teknik Dergi  
2                                     Energy Reports  
3                       IEEE Control Systems Letters  
4                       IEEE Control Systems Letters

Example of multiple keywords: `"Machine Learning"` : this searches for
keywords independently.

In [13]:
%dsldf search publications for "Machine Learning" return publications limit 5

Returned Publications: 5 (total = 2712835)
Time: 0.87s


author_affiliations      pages  \
0  [[{'raw_affiliation': ['Universiti Teknikal Ma...  2650-2657   
1  [[{'raw_affiliation': ['International Universi...     100034   
2  [[{'raw_affiliation': [], 'first_name': 'Görke...        NaN   
3  [[{'raw_affiliation': ['Department of Electric...      43-51   
4  [[{'raw_affiliation': ['Engineering Research C...      70-80   

               id  year     type  \
0  pub.1130334381  2021  article   
1  pub.1133596998  2021  article   
2  pub.1129037745  2021  article   
3  pub.1132780778  2021  article   
4  pub.1132837792  2021  article   

                                               title volume issue  \
0  K-nearest neighbor and naïve Bayes based diagn...      9     6   
1  Blockchain in international e-government proce...      3   NaN   
2  Evaluation of Intersection Properties Using MA...     32     6   
3  Priority-based low voltage DC microgrid system...      7   NaN   
4  A novel optimum arrangement for a hybrid renew...      7   NaN   

     journal.id                                      journal.title  
0  jour.1144063  Bulletin of Electrical Engineering and Informa...  
1  jour.1367388                          Research in Globalization  
2  jour.1226975                                       Teknik Dergi  
3  jour.1150945                                     Energy Reports  
4  jour.1150945                                     Energy Reports

Note: Special characters, such as any of `^ " : ~ \ [ ] { } ( ) ! | & +` must be escaped by a backslash `\`. Also, please note escaping rules in
[Python](http://python-reference.readthedocs.io/en/latest/docs/str/escapes.html) (or other languages). For example, when writing a query with escaped quotes, such as `search publications for "\"phrase 1\" AND \"phrase 2\""`, in Python, it is necessary to escape the backslashes as well, so it
would look like: `'search publications for "\\"phrase 1\\" AND \\"phrase 2\\""'`. 

See the [official docs](https://docs.dimensions.ai/dsl/language.html#for-search-term) for more details.

### 2.3 Boolean Operators

Search term can consist of multiple keywords or phrases connected using
boolean logic operators, e.g. `AND`, `OR` and `NOT`.

In [14]:
%dsldf search publications for "(dose AND concentration)" return publications limit 5

Returned Publications: 5 (total = 5559328)
Time: 1.34s


author_affiliations   pages              id  \
0  [[{'raw_affiliation': ['Department of Pharmace...  100069  pub.1134000145   
1  [[{'raw_affiliation': ['Department of Clinical...  100017  pub.1133580772   
2  [[{'raw_affiliation': ['Chemistry Department, ...  100031  pub.1134186230   
3  [[{'raw_affiliation': ['Section of Molecular H...  100010  pub.1131161031   
4  [[{'raw_affiliation': ['Department of Biotechn...  100018  pub.1133594439   

   year     type                                              title volume  \
0  2021  article  On the relationship between blend state and di...      3   
1  2021  article  Malaria parasites and circadian rhythm: New in...      2   
2  2021  article  Nanocellulose: A mini-review on types and use ...      2   
3  2021  article  PMAP-36 reduces the innate immune response ind...      2   
4  2021  article  Bioprospecting of cowdung microflora for susta...      2   

     journal.id                                      journal.title  
0  jour.1365491           International Journal of Pharmaceutics X  
1  jour.1388648             Current Research in Microbial Sciences  
2  jour.1390798  Carbohydrate Polymer Technologies and Applicat...  
3  jour.1388648             Current Research in Microbial Sciences  
4  jour.1388648             Current Research in Microbial Sciences

When specifying Boolean operators with keywords such as `AND`, `OR` and
`NOT`, the keywords must appear in all uppercase. 

The operators available are shown in the table below.
.

| Boolean Operator | Alternative Symbol | Description                                                                                                                                                                 |
|------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `AND`            | `&&`               | Requires both terms on either side of the Boolean operator to be present for a match.                                                                                       |
| `NOT`            | `!`                | Requires that the following term not be present.                                                                                                                            |
| `OR`             | `||`               | Requires that either term (or both terms) be present for a match.                                                                                                           |
|                  | `+`                | Requires that the following term be present.                                                                                                                                |
|                  | `-`                | Prohibits the following term (that is, matches on fields or documents that do not include that term). The `-` operator is functionally similar to the Boolean operator `!`. |

In [15]:
%dsldf search publications for "(dose OR concentration) AND (-malaria +africa)" return publications limit 5

Returned Publications: 5 (total = 1468851)
Time: 1.07s


author_affiliations    pages              id  \
0  [[{'raw_affiliation': ['Institute of Animal Pr...   100014  pub.1132941183   
1  [[{'raw_affiliation': ['Joint Global Change Re...   100023  pub.1133100081   
2  [[{'raw_affiliation': ['Niger Delta University...     1-15  pub.1133401690   
3                                                NaN      NaN  pub.1107030350   
4  [[{'raw_affiliation': ['National-Regional Join...  387-398  pub.1134184334   

   year     type                                              title volume  \
0  2021  article  A quadruple protection procedure for resuming ...      2   
1  2021  article  Quantifying the reductions in mortality from a...      2   
2  2021  article  Performance Evaluation of Adopting the Electro...      6   
3  2021     book               The Wiley Handbook of Home Education    NaN   
4  2021  article  Root-associated (rhizosphere and endosphere) m...    104   

     journal.id                                      journal.title issue  
0  jour.1388648             Current Research in Microbial Sciences   NaN  
1           NaN                                                NaN   NaN  
2  jour.1355738  International Journal of Applied Research on P...     2  
3           NaN                                                NaN   NaN  
4  jour.1297326                  Journal of Environmental Sciences   NaN

The combination of keywords and boolean operators allow to construct rather sophisticated queries. For example, here's a real-world query used to extract publications related to COVID-19. 

In [16]:
q_inner = """ "2019-nCoV" OR "COVID-19" OR "SARS-CoV-2" OR "HCoV-2019" OR "hcov" OR "NCOVID-19" OR  
    "severe acute respiratory syndrome coronavirus 2" OR "severe acute respiratory syndrome corona virus 2" 
    OR (("coronavirus"  OR "corona virus") AND (Wuhan OR China OR novel)) """

# tip: dsl_escape is a dimcli utility function for escaping special characters 
q_outer = f"""search publications in full_data for "{dsl_escape(q_inner)}" return publications"""
print(q_outer)

dsl.query(q_outer)

search publications in full_data for " \"2019-nCoV\" OR \"COVID-19\" OR \"SARS-CoV-2\" OR \"HCoV-2019\" OR \"hcov\" OR \"NCOVID-19\" OR  
    \"severe acute respiratory syndrome coronavirus 2\" OR \"severe acute respiratory syndrome corona virus 2\" 
    OR ((\"coronavirus\"  OR \"corona virus\") AND (Wuhan OR China OR novel)) " return publications
Returned Publications: 20 (total = 326360)
Time: 2.04s


<dimcli.DslDataset object #4793105568. Records: 20/326360>

### 2.4 Wildcard Searches

The DSL supports single and multiple character wildcard searches within
single terms. Wildcard characters can be applied to single terms, but
not to search phrases.

In [17]:
%dsldf search publications in title_only for "ital? malaria" return publications limit 5

Returned Publications: 5 (total = 148)
Time: 0.96s


id                                              title  year  \
0  pub.1133261890  Artemisinin resistant surveillance in African ...  2020   
1  pub.1132438137  Does Living in Previously Exposed Malaria or W...  2020   
2  pub.1130290794  A Cluster of Cryptic Plasmodium falciparum Mal...  2020   
3  pub.1128245696  Non-imported malaria in Italy: paradigmatic ap...  2020   
4  pub.1124231018  Seasons in Italy: Northern European travelers,...  2020   

                                 author_affiliations     type    journal.id  \
0  [[{'raw_affiliation': ['Department of Infectio...  article  jour.1112262   
1                                                NaN  article  jour.1278986   
2  [[{'raw_affiliation': ['Operative Unit of Infe...  article  jour.1023805   
3  [[{'raw_affiliation': ['Dipartimento Malattie ...  article  jour.1024954   
4  [[{'raw_affiliation': ['Carnegie Mellon Univer...  article  jour.1141817   

                                journal.title volume issue      pages  
0                  Journal of Travel Medicine    NaN   NaN        NaN  
1  Biointerface Research in Applied Chemistry     11     2  9744-9748  
2          Vector-Borne and Zoonotic Diseases     20    12    927-931  
3                           BMC Public Health     20     1        857  
4      Journal of Tourism and Cultural Change    NaN   NaN       1-20

In [18]:
%dsldf search publications in title_only for "it* malaria" return publications limit 5

Returned Publications: 5 (total = 1593)
Time: 0.79s


author_affiliations              id  year  \
0  [[{'raw_affiliation': ['Department of Chemistr...  pub.1133719264  2020   
1  [[{'raw_affiliation': [], 'first_name': 'P. A....  pub.1133730474  2020   
2  [[{'raw_affiliation': ['Department of Parasito...  pub.1133560867  2020   
3  [[{'raw_affiliation': [], 'first_name': 'Zacch...  pub.1133512831  2020   
4  [[{'raw_affiliation': ['Department of Infectio...  pub.1133261890  2020   

      type                                              title volume  \
0  article  S-Adenosyl-L-homocysteine hydrolase: Its Inhib...     21   
1  article  Knowledge of Malaria and Utilization of Its Pr...    NaN   
2  article  Case Report: The First Case of Genotypically C...    NaN   
3  article  Bio-Fabrication of ZnO-CuO Nanoporous Composit...    NaN   
4  article  Artemisinin resistant surveillance in African ...    NaN   

     journal.id                                      journal.title  pages  
0  jour.1029063                Mini-Reviews in Medicinal Chemistry    NaN  
1  jour.1050160  International Journal of TROPICAL DISEASE & He...  34-46  
2  jour.1017021  American Journal of Tropical Medicine and Hygiene    NaN  
3  jour.1050059   Journal of Pharmaceutical Research International  31-39  
4  jour.1112262                         Journal of Travel Medicine    NaN

| Wildcard Search Type                                             | Special Character | Example                                                                                                                                                                                                                         |
|------------------------------------------------------------------|-------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Single character - matches a single character                    | `?`               | The search string `te?t` would match both `test` and `text`.                                                                                                                                                                    |
| Multiple characters - matches zero or more sequential characters | `*`               | The wildcard search: `tes*` would match `test`, `testing`, and `tester`. You can also use wildcard characters in the middle of a term. For example: `te*t` would match `test` and `text`. `*est` would match `pest` and `test`. |

### 2.5 Proximity Searches

A proximity search looks for terms that are within a specific distance
from one another.

To perform a proximity search, add the tilde character `~` and a numeric
value to the end of a search phrase. For example, to search for a
`formal` and `model` within 10 words of each other in a document, use
the search:

In [19]:
%dsldf search publications for "\"formal model\"~10" return publications limit 5

Returned Publications: 5 (total = 504755)
Time: 2.02s


title issue     type  \
0  Formal Analysis of Database Trigger Systems Us...     4  article   
1  Performance Evaluation of Adopting the Electro...     2  article   
2               The Wiley Handbook of Home Education   NaN     book   
3  Uniform parsing for hyperedge replacement gram...   NaN  article   
4  Revisiting positive affect and reward influenc...   NaN  article   

               id                                author_affiliations volume  \
0  pub.1133001578  [[{'raw_affiliation': ['Hanoi University of Mi...      9   
1  pub.1133401690  [[{'raw_affiliation': ['Niger Delta University...      6   
2  pub.1107030350                                                NaN    NaN   
3  pub.1132947815  [[{'raw_affiliation': ['Department of Computin...    118   
4  pub.1134185830  [[{'raw_affiliation': ['Department of Psycholo...     39   

   pages  year    journal.id  \
0   1-16  2021  jour.1148037   
1   1-15  2021  jour.1355738   
2    NaN  2021           NaN   
3   1-27  2021  jour.1124986   
4  27-33  2021  jour.1050888   

                                       journal.title  
0       International Journal of Software Innovation  
1  International Journal of Applied Research on P...  
2                                                NaN  
3            Journal of Computer and System Sciences  
4             Current Opinion in Behavioral Sciences

In [20]:
%dsldf search publications for "\"digital humanities\"~5  +ontology" return publications limit 5

Returned Publications: 5 (total = 10394)
Time: 1.18s


title issue     type  \
0  Participatory Mapping and PGIS: Secerning Fact...     3  article   
1      A neural Entity Coreference Resolution review   NaN  article   
2  Generating knowledge graphs by employing Natur...   NaN  article   
3  Visualizing stemming techniques on online news...     1  article   
4       Ontology learning: Grand tour and challenges   NaN  article   

               id                                author_affiliations volume  \
0  pub.1134005335  [[{'raw_affiliation': ['Centro de Investigacio...     10   
1  pub.1133368057  [[{'raw_affiliation': ['School of Informatics,...    168   
2  pub.1132205206  [[{'raw_affiliation': ['Department of Mathemat...    116   
3  pub.1131619978  [[{'raw_affiliation': ['Universiti Teknologi M...     10   
4  pub.1133579238  [[{'raw_affiliation': ['NLP, Machine Learning ...     39   

     pages  year    journal.id  \
0  105-123  2021  jour.1147177   
1   114466  2021  jour.1128045   
2  253-264  2021  jour.1125399   
3  365-373  2021  jour.1144063   
4   100339  2021  jour.1147019   

                                       journal.title  
0       International Journal of E-Planning Research  
1                   Expert Systems with Applications  
2                 Future Generation Computer Systems  
3  Bulletin of Electrical Engineering and Informa...  
4                            Computer Science Review

The distance referred to here is the number of term movements needed to match the specified phrase.  
In the example above, if `formal` and `model` were 10 spaces apart in a
field, but `formal` appeared before `model`, more than 10 term movements
would be required to move the terms together and position `formal` to
the right of `model` with a space in between.

## 3. Field Searching

Field searching allows to use a specific `field` of a `source` as a
query filter. For example, this can be a
[Literal](supported-types.ipynb) field such as the $type$ of a
publication, its $date$, $mesh terms$, etc.. Or it can be an
[entity](data-entities.ipynb) field, such as the $journal title$ for a
publication, the $country name$ of its author affiliations, etc..

**What are the fields available for each source?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically: 

In [21]:
%dsldocs publications  

sources                       field               type  \
0   publications                    abstract             string   
1   publications            acknowledgements             string   
2   publications                   altmetric              float   
3   publications                altmetric_id            integer   
4   publications                     authors               json   
5   publications                    book_doi             string   
6   publications           book_series_title             string   
7   publications                  book_title             string   
8   publications                category_bra         categories   
9   publications                category_for         categories   
10  publications                category_hra         categories   
11  publications            category_hrcs_hc         categories   
12  publications           category_hrcs_rac         categories   
13  publications           category_icrp_cso         categories   
14  publications            category_icrp_ct         categories   
15  publications               category_rcdc         categories   
16  publications                category_sdg         categories   
17  publications                category_uoa         categories   
18  publications          clinical_trial_ids             string   
19  publications                    concepts               json   
20  publications             concepts_scores               json   
21  publications                        date               date   
22  publications               date_inserted               date   
23  publications              dimensions_url             string   
24  publications                         doi             string   
25  publications        field_citation_ratio              float   
26  publications            funder_countries          countries   
27  publications                     funders      organizations   
28  publications                          id             string   
29  publications                        issn             string   
30  publications                       issue             string   
31  publications                     journal           journals   
32  publications               journal_lists             string   
33  publications                     linkout             string   
34  publications                  mesh_terms             string   
35  publications      open_access_categories        open_access   
36  publications                       pages             string   
37  publications                       pmcid             string   
38  publications                        pmid             string   
39  publications           proceedings_title             string   
40  publications                   publisher             string   
41  publications            recent_citations            integer   
42  publications               reference_ids             string   
43  publications             referenced_pubs  publication_links   
44  publications     relative_citation_ratio              float   
45  publications         research_org_cities             cities   
46  publications      research_org_countries          countries   
47  publications  research_org_country_names             string   
48  publications          research_org_names             string   
49  publications    research_org_state_codes             states   
50  publications    research_org_state_names             string   
51  publications               research_orgs      organizations   
52  publications                 researchers        researchers   
53  publications   resulting_publication_doi             string   
54  publications        supporting_grant_ids             string   
55  publications                 times_cited            integer   
56  publications                       title             string   
57  publications                        type             string   
58  publications                      volume            

### 3.1 `where`

This optional phrase consists of the keyword `where` followed by a
`filters` phrase consisting of DSL filter expressions, as described
below.

In [22]:
%dsldf search publications where type = "book" return publications limit 5

Returned Publications: 5 (total = 445353)
Time: 0.52s


id  year  type                                title
0  pub.1132813748  2021  book  Advances in Hospitality and Leisure
1  pub.1120755114  2021  book      Wege zum Werk als Sinngeschehen
2  pub.1132180633  2021  book                 De animae quantitate
3  pub.1096537870  2021  book                   Handbook of Autism
4  pub.1098573300  2021  book   Metalloproteins and Metalloenzymes

If a `for` phrase is also used in a filtered query, the
system will first apply the filters, and then search the resulting
restricted set of documents for the `search term`.

In [23]:
%dsldf search publications for "malaria" where type = "book" return publications limit 5

Returned Publications: 5 (total = 18783)
Time: 0.55s


volume              id                                              title  \
0    217  pub.1132680288  Viruses and Human Cancer, From Basic Science t...   
1    146  pub.1131964912  Intelligent Computing and Networking, Proceedi...   
2   1255  pub.1132865116  Proceedings of International Conference on Fro...   
3    NaN  pub.1131634515                        Trauma Induced Coagulopathy   
4    NaN  pub.1132931048  Back Pain in the Young Child and Adolescent, A...   

   year  type  
0  2021  book  
1  2021  book  
2  2021  book  
3  2021  book  
4  2021  book

### 3.2 `in`

For convenience, the DSL also supports shorthand notation for filters
where a particular field should be restricted to a specified range or
list of values (although the same logic may be expressed using complex
filters as shown below).

Syntax: a **range filter** consists of the `field` name, the keyword `in`, and a
range of values enclosed in square brackets (`[]`), where the range
consists of a `low` value, colon `:`, and a `high` value.

In [24]:
%%dsldf 
search grants 
    for "malaria" 
    where start_year in [ 2010 : 2015 ] 
return grants limit 5

Returned Grants: 5 (total = 3157)
Time: 0.67s


project_num    end_date                     active_year  \
0  R21AI120981  2017-11-30              [2015, 2016, 2017]   
1  R21AI120973  2019-02-28  [2015, 2016, 2017, 2018, 2019]   
2  R21AI109439  2018-11-30        [2015, 2016, 2017, 2018]   
3        91488  2018-12-18        [2015, 2016, 2017, 2018]   
4          N/A  2019-09-30  [2015, 2016, 2017, 2018, 2019]   

                                      original_title  start_year  \
0  Bloodborne tropical pathogen detection using m...        2015   
1  Field-deployable Assay for Differential Diagno...        2015   
2  T cell driven antigen discovery for vaccine ca...        2015   
3  Senior Fellowship for Dr. Eduardo Samo Gudo: E...        2015   
4  Biology, Ecology & Management of Emerging Dise...        2015   

              id                                   funding_org_name  \
0  grant.4729738  National Institute of Allergy and Infectious D...   
1  grant.4729736  National Institute of Allergy and Infectious D...   
2  grant.4729699  National Institute of Allergy and Infectious D...   
3  grant.4854433                              Volkswagen Foundation   
4  grant.8821176         National Institute of Food and Agriculture   

  title_language language                                              title  \
0             en       en  Bloodborne tropical pathogen detection using m...   
1             en       en  Field-deployable Assay for Differential Diagno...   
2             en       en  T cell driven antigen discovery for vaccine ca...   
3             en       en  Senior Fellowship for Dr. Eduardo Samo Gudo: E...   
4             en       en  Biology, Ecology & Management of Emerging Dise...   

   start_date                                            funders  
0  2015-12-28  [{'id': 'grid.419681.3', 'country_name': 'Unit...  
1  2015-12-24  [{'id': 'grid.419681.3', 'country_name': 'Unit...  
2  2015-12-21  [{'id': 'grid.419681.3', 'country_name': 'Unit...  
3  2015-12-18  [{'id': 'grid.452969.5', 'country_name': 'Germ...  
4  2015-12-10  [{'id': 'grid.482914.2', 'country_name': 'Unit...

Syntax: a **list filter** consists of the `field` name, the keyword `in`, and a list
of one or more `value` s enclosed in square brackets (`[]`), where
values are separated by commas (`,`):

In [25]:
%%dsldf 
search grants 
    for "malaria" 
    where research_org_name in [ "UC Berkeley", "UC Davis", "UCLA"  ] 
return grants limit 5

Returned Grants: 0
Time: 0.70s
WARNINGS [1]
Field 'research_org_name' is deprecated in favor of research_orgs. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


Empty DataFrame
Columns: []
Index: []

### 3.3 `count` - filter function

The filter function `count` is supported on some fields in
[publications](publications.ipynb) (e.g. `researchers` and
`research_orgs`).

Use of this filter is shown on the example below:

In [26]:
%%dsldf 
search publications 
    for "malaria" 
    where count(research_orgs) > 5 
return research_orgs limit 5

Returned Research_orgs: 5
Time: 0.80s


id  count    country_name  \
0    grid.4991.5   1920  United Kingdom   
1    grid.8991.9   1643  United Kingdom   
2  grid.38142.3c   1374   United States   
3  grid.21107.35    900   United States   
4    grid.7445.2    895  United Kingdom   

                                           name   latitude  city_name  \
0                          University of Oxford  51.753437     Oxford   
1  London School of Hygiene & Tropical Medicine  51.520900     London   
2                            Harvard University  42.377052  Cambridge   
3                      Johns Hopkins University  39.328888  Baltimore   
4                       Imperial College London  51.498600     London   

                        linkout  longitude     state_name        types acronym  
0        [http://www.ox.ac.uk/]  -1.254010    Oxfordshire  [Education]     NaN  
1     [http://www.lshtm.ac.uk/]  -0.130700         Camden  [Education]   LSHTM  
2     [http://www.harvard.edu/] -71.116650  Massachusetts  [Education]     NaN  
3        [https://www.jhu.edu/] -76.620280       Maryland  [Education]     JHU  
4  [http://www.imperial.ac.uk/]  -0.175478    Westminster  [Education]     NaN

Number of publications with more than 50 researcher.

In [27]:
%%dsldf 
search publications 
    for "malaria" 
    where count(researchers) > 50 
return publications limit 5

Returned Publications: 5 (total = 212)
Time: 0.80s


title     type              id  \
0  Detection of SARS-CoV-2 N-antigen in blood dur...  article  pub.1133366554   
1  Multiomics Characterization of Preterm Birth i...  article  pub.1133588553   
2                                       Ensembl 2021  article  pub.1131509875   
3  Detection of neutralising antibodies to SARS-C...  article  pub.1131972881   
4  Five insights from the Global Burden of Diseas...  article  pub.1131794493   

                                 author_affiliations  year    journal.id  \
0  [[{'raw_affiliation': ['Université de Paris, I...  2020  jour.1114775   
1  [[{'raw_affiliation': ['Department of Pediatri...  2020  jour.1321825   
2  [[{'raw_affiliation': ['European Molecular Bio...  2020  jour.1018982   
3  [[{'raw_affiliation': ['Department of Zoology,...  2020  jour.1021032   
4  [[{'raw_affiliation': [], 'first_name': 'GBD 2...  2020  jour.1077219   

                         journal.title  issue volume      pages  
0  Clinical Microbiology and Infection    NaN    NaN        NaN  
1                    JAMA Network Open     12      3   e2029655  
2               Nucleic Acids Research    NaN    NaN   gkaa942-  
3                     Eurosurveillance     42     25    2000685  
4                           The Lancet  10258    396  1135-1159

Number of publications with more than one researcher.

In [28]:
%%dsldf 
search publications
where count(researchers) > 1
return funders limit 5

Returned Funders: 5
Time: 1.97s


id    count   country_name acronym   longitude         types  \
0  grid.419696.5  1979524          China    NSFC  116.339830  [Government]   
1  grid.270680.b   725673        Belgium      EC    4.363670  [Government]   
2  grid.424020.0   622277          China    MOST  116.316284  [Government]   
3  grid.54432.34   570374          Japan    JSPS  139.740390   [Nonprofit]   
4  grid.48336.3a   559674  United States     NCI  -77.101190  [Government]   

   city_name                                    linkout  \
0    Beijing  [http://www.nsfc.gov.cn/publish/portal1/]   
1   Brussels         [http://ec.europa.eu/index_en.htm]   
2    Beijing              [http://www.most.gov.cn/eng/]   
3      Tokyo                   [http://www.jsps.go.jp/]   
4  Rockville                   [http://www.cancer.gov/]   

                                                name   latitude state_name  
0       National Natural Science Foundation of China  40.005177        NaN  
1                                European Commission  50.851650        NaN  
2  Ministry of Science and Technology of the Peop...  39.827835        NaN  
3         Japan Society for the Promotion of Science  35.687160        NaN  
4                          National Cancer Institute  39.004326   Maryland

International collaborations: number of publications with more than one author and affiliations located in more than one country.

In [29]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) > 1
return funders limit 5

Returned Funders: 5
Time: 1.01s


id   count country_name  \
0  grid.419696.5  481660        China   
1  grid.270680.b  371757      Belgium   
2  grid.424150.6  164661      Germany   
3  grid.424020.0  156442        China   
4  grid.54432.34  145633        Japan   

                                                name   latitude city_name  \
0       National Natural Science Foundation of China  40.005177   Beijing   
1                                European Commission  50.851650  Brussels   
2                         German Research Foundation  50.699340      Bonn   
3  Ministry of Science and Technology of the Peop...  39.827835   Beijing   
4         Japan Society for the Promotion of Science  35.687160     Tokyo   

                                     linkout   longitude acronym         types  
0  [http://www.nsfc.gov.cn/publish/portal1/]  116.339830    NSFC  [Government]  
1         [http://ec.europa.eu/index_en.htm]    4.363670      EC  [Government]  
2                    [http://www.dfg.de/en/]    7.147797     DFG    [Facility]  
3              [http://www.most.gov.cn/eng/]  116.316284    MOST  [Government]  
4                   [http://www.jsps.go.jp/]  139.740390    JSPS   [Nonprofit]

Domestic collaborations: number of publications with more than one author and more than one affiliation located in exactly one country.

In [30]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) = 1
return funders limit 5

Returned Funders: 5
Time: 2.70s


id    count   country_name  \
0  grid.419696.5  1455454          China   
1  grid.424020.0   454801          China   
2  grid.48336.3a   409234  United States   
3  grid.54432.34   390452          Japan   
4  grid.270680.b   336357        Belgium   

                                                name   latitude  city_name  \
0       National Natural Science Foundation of China  40.005177    Beijing   
1  Ministry of Science and Technology of the Peop...  39.827835    Beijing   
2                          National Cancer Institute  39.004326  Rockville   
3         Japan Society for the Promotion of Science  35.687160      Tokyo   
4                                European Commission  50.851650   Brussels   

                                     linkout   longitude acronym  \
0  [http://www.nsfc.gov.cn/publish/portal1/]  116.339830    NSFC   
1              [http://www.most.gov.cn/eng/]  116.316284    MOST   
2                   [http://www.cancer.gov/]  -77.101190     NCI   
3                   [http://www.jsps.go.jp/]  139.740390    JSPS   
4         [http://ec.europa.eu/index_en.htm]    4.363670      EC   

          types state_name  
0  [Government]        NaN  
1  [Government]        NaN  
2  [Government]   Maryland  
3   [Nonprofit]        NaN  
4  [Government]        NaN

### 3.4 Filter Operators

A simple filter expression consists of a `field` name, an in-/equality
operator `op`, and the desired field `value`. 

The `value` must be a
`string` enclosed in double quotes (`"`) or an integer (e.g. `1234`).

The available operators are:

| `op`           | meaning                                                                                  |
|----------------|------------------------------------------------------------------------------------------|
| `=`            | *is* (or *contains* if the given `field` is multi-value)                                 |
| `!=`           | *is not*                                                                                 |
| `>`            | *is greater than*                                                                        |
| `<`            | *is less than*                                                                           |
| `>=`           | *is greater than or equal to*                                                            |
| `<=`           | *is less than or equal to*                                                               |
| `~`            | *partially matches* (see partial-string-matching below) |
| `is empty`     | *is empty* (see emptiness-filters below)                      |
| `is not empty` | *is not empty* (see emptiness-filters below)                  |

A couple of examples 

In [31]:
%dsldf search datasets where year > 2010 and year < 2012 return datasets limit 5

Returned Datasets: 5 (total = 155827)
Time: 0.61s


id                                           keywords  year  \
0  dataset.8857124  [Composable Chemiosmotic Energy Transduction R...  2011   
1  dataset.8861504  [tandem mass spectrometry, bile duct obstructi...  2011   
2  dataset.8857589      [Superantigen Toxicity Clarifying, Mechanism]  2011   
3  dataset.8861444  [tandem mass spectrometry, bile duct obstructi...  2011   
4  dataset.8857670      [Superantigen Toxicity Clarifying, Mechanism]  2011   

                                               title  \
0                    Electron Transport Chain (ETC).   
1  Relative abundance of taurine- (A) and glycine...   
2  A cartoon of what might be happening at the im...   
3  Relative abundance of primary (A & D), seconda...   
4  The model structures of four trimer complexes ...   

                                             authors    journal.id  \
0  [{'name': 'Ivan Chang'}, {'name': 'Margit Heis...  jour.1037553   
1  [{'name': 'Jocelyn Trottier'}, {'name': 'Andrz...  jour.1037553   
2                       [{'name': 'John D. Fraser'}]  jour.1032549   
3  [{'name': 'Jocelyn Trottier'}, {'name': 'Andrz...  jour.1037553   
4                       [{'name': 'John D. Fraser'}]  jour.1032549   

  journal.title  
0      PLoS ONE  
1      PLoS ONE  
2  PLOS Biology  
3      PLoS ONE  
4  PLOS Biology

In [32]:
%dsldf search patents where assignees != "grid.410484.d" return patents limit 5

Returned Patents: 5 (total = 54549156)
Time: 0.84s


publication_date                id  \
0       1958-11-13      DE-1043324-B   
1       2011-03-17  WO-2011032015-A1   
2       2012-05-08     US-8173093-B2   
3       2004-03-01       CA-100082-S   
4       1971-01-14     DE-1935222-A1   

                                               title  year  times_cited  \
0  A method for the isomerization of 4substituier...  1954            0   
1  CONCURRENT WIRELESS TRANSMITTER MAPPING AND MO...  2010            0   
2  Iron silicide sputtering target and method for...  2005            2   
3                                             STATUE  2002            0   
4  Device for automatic speiseresteabraeumung in ...  1969            0   

                                      inventor_names  \
0  [BAIN JOSEPH PAUL, KLEIN EUGENE ALBERT, HUNT H...   
1  [GARIN LIONEL JACQUES, DO JU-YONG, ZHANG GENGS...   
2                         [ODA KUNIHIRO, SUZUKI RYO]   
3                                                NaN   
4                                  [P CONRADI BENNO]   

                                      assignee_names filing_status  \
0                [Glidden Co, Akzo Nobel Paints LLC]   Application   
1  [GARIN LIONEL JACQUES, QUALCOMM INC, DO JU-YON...   Application   
2                 [JX Nippon Mining and Metals Corp]         Grant   
3               [Henri Studio Inc, HENRI STUDIO INC]         Grant   
4                                    [KUEPPERSBUSCH]   Application   

                                           assignees  granted_year  
0                                                NaN           NaN  
1  [{'id': 'grid.430388.4', 'country_name': 'Unit...           NaN  
2  [{'id': 'grid.497092.1', 'country_name': 'Japa...        2012.0  
3                                                NaN        2004.0  
4                                                NaN           NaN

### 3.5 Partial string matching with `~`

The `~` operator indicates that the given `field` need only partially,
instead of exactly, match the given `string` (the `value` used with this
operator must be a `string`, not an integer).

For example, the filter `where research_orgs.name~"Saarland Uni"` would
match both the organization named "Saarland University" and the one
named "Universitätsklinikum des Saarlandes", and any other organization
whose name includes the terms "Saarland" and "Uni" (the order is
unimportant). 

In [33]:
%%dsldf 
search patents 
    where assignee_names ~ "IBM" 
return assignees limit 5

Returned Assignees: 5
Time: 2.42s


id  count    country_name                  name   latitude  \
0  grid.410484.d  72795   United States   IBM (United States)  41.108540   
1  grid.14648.3f   5362  United Kingdom  IBM (United Kingdom)  51.026752   
2  grid.424815.e   3377         Germany         IBM (Germany)  48.673832   
3  grid.424192.8   1499          France          IBM (France)  48.843975   
4  grid.292504.8   1348          Canada          IBM (Canada)  43.819103   

    city_name                       linkout  longitude state_name      types  
0      Armonk         [http://www.ibm.com/] -73.720470   New York  [Company]  
1  Winchester   [https://www.ibm.com/in-en]  -1.397260  Hampshire  [Company]  
2   Böblingen   [http://www.ibm.com/de/de/]   9.034824        NaN  [Company]  
3       Paris  [https://www.ibm.com/fr-fr/]   2.396280        NaN  [Company]  
4     Markham   [http://www.ibm.com/ca/en/] -79.333930    Ontario  [Company]

### 3.6 Emptiness filters `is empty`

To filter records which contain specific field or to filter those which
contain an empty field, it is possible to use something like
`where research_orgs is not empty` or `where issn is empty`.

In [34]:
%%dsldf
search publications 
    for "iron graphene" 
    where researchers is empty 
    and research_orgs is not empty 
return publications[id+title+researchers+research_orgs+type] limit 5

Returned Publications: 5 (total = 3989)
Time: 1.42s


title  \
0  Algal-based polysaccharides as polymer electro...   
1  Antimicrobial edible films in food packaging: ...   
2  The application of nanoparticles in cancer imm...   
3  Influence of drying and calcination temperatur...   
4  Research progress and mechanism of nanomateria...   

                                       research_orgs     type              id  
0  [{'id': 'grid.441908.0', 'country_name': 'Peru...  article  pub.1133689836  
1  [{'id': 'grid.411890.5', 'country_name': 'Indi...  article  pub.1133731113  
2  [{'id': 'grid.412523.3', 'country_name': 'Chin...  article  pub.1134129893  
3  [{'id': 'grid.163032.5', 'country_name': 'Chin...  article  pub.1133997670  
4  [{'id': 'grid.410726.6', 'country_name': 'Chin...  article  pub.1134184045

## 4. Searching for Researchers

The DSL offers different mechanisms for searching for researchers (e.g.
publication authors, grant investigators), each of them presenting
specific advantages.

### 4.1 Exact name searches

Special full-text indices allows to look up a researcher's name and
surname **exactly as they appear in the source documents** they derive from.

This approach has a broad scope, as it allows to search the full
collection of Dimensions documents irrespectively of whether a
researcher was succesfully disambiguated (and hence given a Dimensions
ID). On the other hand, this approach will only match names as they
appear in the source document, so different spellings or initials are
not necessarily returned via a single query. 

```
search in [authors|investigators|inventors]
```

It is possible to look up publications authors using a specific
`search index` called `authors`. 

This method expects case insensitive
phrases, in format $"<first name> <last name>"$ or reverse order. Note
that strings in double quotes that contain nested quotes must always be
escaped by a backslash `\`.

In [35]:
%dsldf search publications in authors for "\"Charles Peirce\"" return publications limit 5

Returned Publications: 5 (total = 145)
Time: 0.58s


author_affiliations    pages              id  \
0  [[{'raw_affiliation': [], 'first_name': 'Charl...    37-49  pub.1132626305   
1  [[{'raw_affiliation': [], 'first_name': 'Charl...  304-330  pub.1123488526   
2  [[{'raw_affiliation': [], 'first_name': 'Charl...  331-347  pub.1123488527   
3  [[{'raw_affiliation': [], 'first_name': 'Charl...  262-267  pub.1123488522   
4  [[{'raw_affiliation': [], 'first_name': 'Charl...  477-488  pub.1123488535   

   year     type                                   title  
0  2020  chapter             How to Make our Ideas Clear  
1  2019  chapter  10. [Six Papers on Existential Graphs]  
2  2019  chapter           11. On Existential Graphs, F4  
3  2019  chapter        6. Positive Logical Graphs (PLG)  
4  2019  chapter    19. [A System of Existential Graphs]

Instead of first name, initials can also be used. These are examples of
valid research search phrases:

-   `\"Peirce, Charles S.\"`
-   `\"Charles S. Peirce\"`
-   `\"CS Peirce\"`
-   `\"Peirce CS\"`
-   `\"C S Peirce\"`
-   `\"Peirce C S\"`
-   `\"C Peirce\"`
-   `\"Peirce C\"`
-   `\"Charles Peirce\"`
-   `\"Peirce Charles\"`

**Warning**: In order to produce valid results an author or an investigator search
query must contain **at least two components or more** (e.g., name and
surname, either in full or initials).

Investigators search is similar to *authors* search, only it allows to search on `grants` and
`clinical trials` using a separate search index `investigators`, and on
`patents` using the index `inventors`.

In [36]:
%%dsldf 
search clinical_trials in investigators for "\"John Smith\"" 
return clinical_trials limit 5

Returned Clinical_trials: 3 (total = 3)
Time: 0.84s


id                                              title  \
0  NCT00689533  VEPTR Implantation to Treat Children With Earl...   
1  NCT01241149  Prospective Evaluation of Symptom Resolution i...   
2  NCT04072380  A Phase 2, Double-blind, Placebo-controlled, P...   

                                investigator_details  \
0  [[John M Flynn, MD, Principal Investigator, Ch...   
1  [[Ellie Mentler, MD, Principal Investigator, U...   
2  [[Rohith G. Patel, MD, Principal Investigator,...   

                                        active_years  
0  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...  
1                                                NaN  
2                                       [2019, 2020]

In [37]:
%%dsldf 
search grants in investigators for "\"Satoko Shimazaki\"" 
return grants limit 5

Returned Grants: 4 (total = 4)
Time: 0.55s


title  start_year    end_date  \
0  Kabuki Actors, Print Technology, and the Theat...        2021  2022-08-31   
1  Genealogy research on female saints in the Pal...        2018  2021-03-31   
2  Images of Women in the Old English Lives of Sa...        2015  2018-03-31   
3  Reception and Transfromation of the Images of ...        2012  2015-03-31   

   start_date             id                            funding_org_name  \
0  2021-09-01  grant.7925589       National Endowment for the Humanities   
1  2018-04-01  grant.7527261  Japan Society for the Promotion of Science   
2  2015-04-01  grant.5858713  Japan Society for the Promotion of Science   
3  2012-04-01  grant.6086985  Japan Society for the Promotion of Science   

  language               active_year  \
0       en              [2021, 2022]   
1       ja  [2018, 2019, 2020, 2021]   
2       en  [2015, 2016, 2017, 2018]   
3       en  [2012, 2013, 2014, 2015]   

                                      original_title title_language  \
0  Kabuki Actors, Print Technology, and the Theat...             en   
1           古・中英語期における女性聖人伝の系譜研究：Aelfricのテクストと言語を中心に             ja   
2  Images of Women in the Old English Lives of Sa...             en   
3  Reception and Transfromation of the Images of ...             en   

     project_num                                            funders  
0  FEL-263245-19  [{'id': 'grid.422239.c', 'country_name': 'Unit...  
1       18K00431  [{'id': 'grid.54432.34', 'country_name': 'Japa...  
2       15K02313  [{'id': 'grid.54432.34', 'country_name': 'Japa...  
3       24520310  [{'id': 'grid.54432.34', 'country_name': 'Japa...

In [38]:
%%dsldf 
search patents in inventors for "\"John Smith\"" 
return patents limit 5

Returned Patents: 5 (total = 636)
Time: 0.69s


title assignee_names  \
0  Improvements in or relating to Electric Arc La...   [SMITH JOHN]   
1  Improvements in or relating to Steam-heated Co...   [SMITH JOHN]   
2       An Improved Method of Packing Glass Bottles.   [SMITH JOHN]   
3  Improvements in or connected with "Otter Board...   [SMITH JOHN]   
4              IMPROVEMENTS IN STOVE AND FIRE GRATES  [SMITH, JOHN]   

               id publication_date filing_status inventor_names    year  \
0  GB-189625600-A       1897-09-25   Application   [SMITH JOHN]  1896.0   
1  GB-190921910-A       1910-07-28   Application   [SMITH JOHN]  1909.0   
2  GB-189814759-A       1899-07-05   Application   [SMITH JOHN]  1898.0   
3  GB-190114799-A       1902-05-29   Application   [SMITH JOHN]  1901.0   
4       CA-4544-A       1875-04-01         Grant   [SMITH JOHN]     NaN   

   times_cited  granted_year  
0            0           NaN  
1            0           NaN  
2            0           NaN  
3            0           NaN  
4            0        1875.0

### 4.2 Fuzzy Searches

This type of search is similar to *full-text
search*, with the difference that it
allows searching by only a part of a name, e.g. only the 'last name' of
a person, by using the `where` clause. 

**Note** At this moment, this type of search is only available for
`publications`. Other sources will add this option in the future.

For example:

In [71]:
%%dsldf 
search publications where authors = "Hawking" 
return publications[id+doi+title+authors] limit 5

Returned Publications: 5 (total = 1979)
Time: 2.48s


title  \
0  A search for the dimuon decay of the Standard ...   
1  Measurement of the jet mass in high transverse...   
2  Observation and Measurement of Forward Proton ...   
3  Search for Heavy Resonances Decaying into a Ph...   
4  The Influence of Glacial Cover on Riverine Sil...   

                                             authors              id  \
0  [{'raw_affiliation': ['CPPM, Aix-Marseille Uni...  pub.1133087947   
1  [{'raw_affiliation': ['CPPM, Aix-Marseille Uni...  pub.1133086120   
2  [{'raw_affiliation': ['CPPM, Aix-Marseille Uni...  pub.1134007432   
3  [{'raw_affiliation': ['CPPM, Aix-Marseille Uni...  pub.1133603040   
4  [{'raw_affiliation': ['Bristol Glaciology Cent...  pub.1132789280   

                              doi  
0  10.1016/j.physletb.2020.135980  
1  10.1016/j.physletb.2020.135991  
2  10.1103/physrevlett.125.261801  
3  10.1103/physrevlett.125.251802  
4            10.1029/2020gb006611

Generally speaking, using a `where` clause to search authors is less
precise that using the relevant exact-search syntax. 

On the other hand, using a
`where` clause can be handy if one wants to **combine an author search
with another full-text search index**.

For example:

In [40]:
%%dsldf 
search publications 
    in title_abstract_only for "dna replication" 
    where authors = "smith"  
return publications limit 5

Returned Publications: 5 (total = 1552)
Time: 0.79s


title     type              id  \
0  Epigenome-wide association study of diet quali...  article  pub.1131341540   
1  High Risk α-HPV E6 Impairs Translesion Synthes...  article  pub.1134030384   
2  Haloferax volcanii—a model archaeon for studyi...  article  pub.1133027047   
3  Rapid poxvirus engineering using CRISPR/Cas9 a...  article  pub.1132270279   
4  Association between Breastfeeding and DNA Meth...  article  pub.1132201653   

                                 author_affiliations  year    journal.id  \
0  [[{'raw_affiliation': ['Hubert Department of G...  2020  jour.1087544   
1  [[{'raw_affiliation': ['Division of Biology, K...  2020  jour.1043163   
2  [[{'raw_affiliation': ["School of Life Science...  2020  jour.1046605   
3  [[{'raw_affiliation': ['John Curtin School of ...  2020  jour.1300829   
4  [[{'raw_affiliation': ['Postgraduate Programme...  2020  jour.1042723   

                           journal.title issue volume   pages  
0  International Journal of Epidemiology   NaN    NaN     NaN  
1                                Cancers     1     13      28  
2                           Open Biology    12     10  200293  
3                 Communications Biology     1      3     643  
4                              Nutrients    11     12    3309

### 4.3 Using the disambiguated Researchers database

The Dimensions [Researchers](https://docs.dimensions.ai/dsl/datasource-researchers.html) source is a database of
researchers information algorithmically extracted and disambiguated from
all of the other content sources (publications, grants, clinical trials
etc..).

By using the `researchers` source it is possible to match an
'aggregated' person object linking together multiple publication
authors, grant investigators etc.. irrespectively of the form their
names can take in the original source documents.

However, since database does not contain all authors and investigators information
available in Dimensions. 

E.g. think of authors from older publications,
or authors with very common names that are difficult to disambiguate, or
very new authors, who have only one or few publications. In such cases,
using full-text authors search might be more
appropriate.

Examples:

In [41]:
%%dsldf 
search researchers for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 4 (total = 4)
Time: 0.60s


research_orgs  last_name first_name  \
0  [{'id': 'grid.266190.a', 'country_name': 'Unit...  Shimazaki     Satoko   
1  [{'id': 'grid.19006.3e', 'country_name': 'Unit...  Shimazaki     Satoko   
2                                                NaN  Shimazaki     Satoko   
3                                                NaN  Shimazaki     Satoko   

   obsolete                  id  
0         0  ur.015527473602.63  
1         0  ur.014307627665.09  
2         0   ur.07751146721.59  
3         1  ur.010537333602.30

NOTE pay attentiont to the `obsolete` field. This indicates the researcher ID status. 0 means that the researcher ID is still **active**, 1 means that the researcher ID is **no longer valid**. This is due to the ongoing process of refinement of Dimensions researchers. 

Hence the query above is best written like this:

In [42]:
%%dsldf 
search researchers where obsolete=0 for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 3 (total = 3)
Time: 1.10s


research_orgs  last_name first_name  \
0  [{'id': 'grid.266190.a', 'country_name': 'Unit...  Shimazaki     Satoko   
1  [{'id': 'grid.19006.3e', 'country_name': 'Unit...  Shimazaki     Satoko   
2                                                NaN  Shimazaki     Satoko   

   obsolete                  id  
0         0  ur.015527473602.63  
1         0  ur.014307627665.09  
2         0   ur.07751146721.59

With `Researchers`, one can use other fields as well:

In [43]:
%%dsldf 
search researchers 
    where obsolete=0 and last_name="Shimazaki" 
return researchers[basics] limit 5

Returned Researchers: 5 (total = 460)
Time: 0.58s


last_name  first_name                  id  \
0  Shimazaki  Ken-Ichiro   ur.07735103434.64   
1  Shimazaki      Taishi  ur.014464167544.15   
2  Shimazaki   Katsunori  ur.012676375026.21   
3  Shimazaki       Kohei  ur.012106470617.55   
4  Shimazaki      Michio  ur.012132522165.33   

                                       research_orgs  
0                                                NaN  
1  [{'id': 'grid.39158.36', 'country_name': 'Japa...  
2                                                NaN  
3                                                NaN  
4                                                NaN

## 5. Returning results

After the `search` phrase, a query must contain one or more `return`
phrases, specifying the content and format of the information that
should be returned.



### 5.1 Returning Multiple Sources

Multiple results may not be returned in a single `return` phrase.

In [44]:
%%dsldf 
search publications 
return funders limit 5 
return research_orgs limit 5 
return year

Returned Research_orgs: 5
Returned Year: 20
Returned Funders: 5
Time: 3.07s
[Warning] Dataframe created from first available key, but more than one JSON key found: ['research_orgs', 'year', 'funders']


id   count   country_name acronym  longitude        types  \
0  grid.26999.3d  427017          Japan      UT  139.76222  [Education]   
1  grid.38142.3c  401733  United States     NaN  -71.11665  [Education]   
2  grid.17063.33  305886         Canada     NaN  -79.39500  [Education]   
3  grid.214458.e  272655  United States      UM  -83.73822  [Education]   
4  grid.258799.8  269874          Japan     NaN  135.77979  [Education]   

   city_name                         linkout                    name  \
0      Tokyo  [http://www.u-tokyo.ac.jp/en/]     University of Tokyo   
1  Cambridge       [http://www.harvard.edu/]      Harvard University   
2    Toronto       [http://www.utoronto.ca/]   University of Toronto   
3  Ann Arbor        [https://www.umich.edu/]  University of Michigan   
4      Kyoto   [http://www.kyoto-u.ac.jp/en]        Kyoto University   

    latitude     state_name  
0  35.713333            NaN  
1  42.377052  Massachusetts  
2  43.661667        Ontario  
3  42.278305       Michigan  
4  35.026157            NaN


### 5.2 Returning Specific Fields

For control over which information from each given `record` will be
returned, a `source` or `entity` name in the `results` phrase can be
optionally followed by a specification of `fields` and `fieldsets` to be
included in the JSON results for each retrieved record.

The fields specification may be an arbitrary list of `field` names
enclosed in brackets (`[`, `]`), with field names separated by a plus
sign (`+`). Minus sign (`-`) can be used to exclude `field` or a
`fieldset` from the result. Field names thus listed within brackets must
be "known" to the DSL, and therefore only a subset of fields may be used
in this syntax (see note below).

In [45]:
%%dsldf 
search grants 
return grants[grant_number + title + language] limit 5

Returned Grants: 5 (total = 5623964)
Time: 0.52s


grant_number                                              title language
0          890218  Functional analysis of ribosome heterogeneity ...       en
1  2018-HRSI-1548  APPROACH to Enriching the Real World Evidence ...       en
2          948141  Simulating ultracold correlated quantum matter...       en
3          887019  The role of microbial Oxylipins in the MIcrobe...       en
4    AH/V001841/1  Playgrounds: A Material Cultural Study of Post...       en

In [46]:
%%dsldf 
search clinical_trials 
return clinical_trials [id+ title + acronym + phase] limit 5

Returned Clinical_trials: 5 (total = 609412)
Time: 0.52s


id phase                                              title  \
0  KCT0002490   N/A  Effect of Vitamin D Supplementation on the Phy...   
1  KCT0002491   N/A  The median effective dose (ED50) of intravenou...   
2  KCT0002492   N/A  Efficacy and Safety of Chrysanthemum indicum e...   
3  KCT0002493   N/A  The effects of thermal softening of single-lum...   
4  KCT0002494   N/A  Efficacy of combined of conventional rehabilit...   

     acronym  
0        NaN  
1        NaN  
2  WONP_CIEE  
3        NaN  
4        NaN

**Shortcuts: `fieldsets`**

The fields specification may be the name of a pre-defined `fieldset`
(e.g. `extras`, `basics`). These are shortcuts that can be handy when testing out new queries, for example. 

NOTE In general when writing code used in integrations or long-standing extraction scripts it is **best to return specific fields rather that a predefined set**. This has also the advantage of making queries faster by avoiding the extraction of unnecessary data.
    

In [47]:
%%dsldf 
search grants 
return grants [basics] limit 5 

Returned Grants: 5 (total = 5623964)
Time: 0.76s
WARNINGS [2]
Field 'project_num' is deprecated in favor of grant_number. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'title_language' is deprecated in favor of language_title. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


funding_org_name  \
0                       European Commission   
1  New Brunswick Health Research Foundation   
2                 European Research Council   
3                       European Commission   
4      Arts and Humanities Research Council   

                                      original_title  start_year  \
0  Functional analysis of ribosome heterogeneity ...        2021   
1  APPROACH to Enriching the Real World Evidence ...        2021   
2  Simulating ultracold correlated quantum matter...        2021   
3  The role of microbial Oxylipins in the MIcrobe...        2021   
4  Playgrounds: A Material Cultural Study of Post...        2021   

                                             funders     project_num  \
0  [{'id': 'grid.270680.b', 'country_name': 'Belg...          890218   
1  [{'id': 'grid.484521.e', 'country_name': 'Cana...  2018-HRSI-1548   
2  [{'id': 'grid.452896.4', 'country_name': 'Belg...          948141   
3  [{'id': 'grid.270680.b', 'country_name': 'Belg...          887019   
4  [{'id': 'grid.426413.6', 'country_name': 'Unit...    AH/V001841/1   

                                               title  start_date language  \
0  Functional analysis of ribosome heterogeneity ...  2021-12-01       en   
1  APPROACH to Enriching the Real World Evidence ...  2021-11-30       en   
2  Simulating ultracold correlated quantum matter...  2021-11-01       en   
3  The role of microbial Oxylipins in the MIcrobe...  2021-11-01       en   
4  Playgrounds: A Material Cultural Study of Post...  2021-10-03       en   

                            active_year title_language             id  \
0                    [2021, 2022, 2023]             en  grant.9064785   
1                                [2021]             en  grant.8690978   
2  [2021, 2022, 2023, 2024, 2025, 2026]             en  grant.9414093   
3                    [2021, 2022, 2023]             en  grant.8964187   
4                    [2021, 2022, 2023]             en  grant.9401634   

     end_date  
0  2023-11-30  
1         NaN  
2  2026-10-31  
3  2023-10-31  
4  2023-04-02

In [48]:
%%dsldf 
search publications 
return publications [basics+times_cited] limit 5 

Returned Publications: 5 (total = 115232311)
Time: 1.52s
WARNINGS [1]
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


id  year                                              title  \
0  pub.1134028727  2021  8 Gender Violence in the Hospitality Industry:...   
1  pub.1126934999  2021  Mosaic Defects of AlN Buffer Layers in GaN/AlN...   
2  pub.1134028720  2021  1 The Perils of Sex Work in Montreal: Seeking ...   
3  pub.1134154735  2021  Gövde Buruşmalarının Elastik Olmayan Yanal Bur...   
4  pub.1131456225  2021      Mağaza İmajının İkon Marka Algısına Yansıması   

     pages  times_cited     type  \
0  184-198            0  chapter   
1      1-1            0  article   
2    16-41            0  chapter   
3      1-1            0  article   
4      NaN            0  article   

                                 author_affiliations    journal.id  \
0                                                NaN           NaN   
1  [[{'raw_affiliation': [], 'first_name': 'Tuğçe...  jour.1152664   
2                                                NaN           NaN   
3  [[{'raw_affiliation': [], 'first_name': 'Mehme...  jour.1152664   
4  [[{'raw_affiliation': [], 'first_name': 'Yelda...  jour.1359387   

                            journal.title  
0                                     NaN  
1                  Journal of Polytechnic  
2                                     NaN  
3                  Journal of Polytechnic  
4  Türkiye İletişim Araştırmaları Dergisi

The fields specification may be an (`all`), to indicate that all fields
available for the given `source` should be returned.

In [ ]:
%%dsldf
search publications 
return publications [all] limit 5 

### 5.3 Returning Facets

In addition to returning source records matching a query, it is possible
to $facet$ on the [entity](data-entities.ipynb) fields related to a
particular source and return only those entity values as an aggregrated
view of the related source data. This operation is similar to a
$group by$ or $pivot table$.

**Warning** Faceting can return up to a maximum of 1000 results. This is to ensure
adequate performance with all queries. Furthemore, although the `limit`
operator is allowed, the `skip` operator cannot be used.

In [50]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_orgs limit 5

Returned Research_orgs: 5
Time: 0.66s


id  count    country_name  \
0  grid.38142.3c   2862   United States   
1  grid.21107.35   1793   United States   
2    grid.4991.5   1765  United Kingdom   
3  grid.17063.33   1687          Canada   
4  grid.33199.31   1591           China   

                                            name   latitude  city_name  \
0                             Harvard University  42.377052  Cambridge   
1                       Johns Hopkins University  39.328888  Baltimore   
2                           University of Oxford  51.753437     Oxford   
3                          University of Toronto  43.661667    Toronto   
4  Huazhong University of Science and Technology  30.508183      Wuhan   

                         linkout  longitude     state_name        types  \
0      [http://www.harvard.edu/]  -71.11665  Massachusetts  [Education]   
1         [https://www.jhu.edu/]  -76.62028       Maryland  [Education]   
2         [http://www.ox.ac.uk/]   -1.25401    Oxfordshire  [Education]   
3      [http://www.utoronto.ca/]  -79.39500        Ontario  [Education]   
4  [http://english.hust.edu.cn/]  114.41474            NaN  [Education]   

  acronym  
0     NaN  
1     JHU  
2     NaN  
3     NaN  
4    HUST

In [51]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_org_countries limit 5
return year limit 5
return category_for limit 5

Returned Research_org_countries: 5
Returned Category_for: 5
Returned Year: 5
Time: 0.57s
[Warning] Dataframe created from first available key, but more than one JSON key found: ['research_org_countries', 'category_for', 'year']


id  count            name
0  US  61172   United States
1  CN  25265           China
2  GB  20635  United Kingdom
3  IT  12273           Italy
4  DE  11149         Germany

For control over the organization and headers of the JSON query results,
the `return` keyword in a return phrase may be followed by the keyword
`in` and then a `group` name for this group of results, where the group
name is enclosed in double quotes(`"`).

Also, one can define `aliases` that replace the defaul JSON fields names with other ones provided by the user. 

See the [official documentation](https://docs.dimensions.ai/dsl/language.html#aliases) for more details about this feature. 

In [52]:
%%dsl
search publications 
return in "facets" funders 
return in "facets" research_orgs

Returned Facets: 2
Time: 2.80s


<dimcli.DslDataset object #4795823104. Records: 2/115232310>

### 5.4 What the query statistics refer to - sources VS facets

When performing a DSL search, a `_stats` object is return which contains some useful info eg the total number of records available for a search. 

In [53]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return publications limit 5

Returned Publications: 5 (total = 5077)
Time: 0.55s


id                                              title  year  \
0  pub.1113308928  A Hybrid DCT-CLAHE Approach for Brightness Enh...  2018   
1  pub.1110958161  Optimized coordinated control of LFC and SMES ...  2018   
2  pub.1110012351  The Role of Lanthanum in a Nickel Oxide‐Based ...  2018   
3  pub.1110932965  Electrostatic Discharge Threshold on Coverglas...  2018   
4  pub.1110925389  Nuclear Ab Initio Calculations with the Unitar...  2018   

                                 author_affiliations      pages        type  \
0  [[{'raw_affiliation': ['Department of Electric...    123-127  proceeding   
1  [[{'raw_affiliation': ['Department of Electric...         39     article   
2  [[{'raw_affiliation': ['Graduate School of Lif...    518-526     article   
3  [[{'raw_affiliation': ['Department of Mechanic...  1445-1452     article   
4  [[{'raw_affiliation': ['Center for Nuclear Stu...        NaN  proceeding   

  volume issue    journal.id                                   journal.title  
0    NaN   NaN           NaN                                             NaN  
1      3     1  jour.1157179  Protection and Control of Modern Power Systems  
2     12     2  jour.1297486                                     ChemSusChem  
3     47     2  jour.1031080             IEEE Transactions on Plasma Science  
4    NaN   NaN           NaN                                             NaN



It is important to note though that the **total number always refers to the main source, never the facets** one is searching for. 

For example, in this query we return `researchers` linked to publications: 

In [54]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return researchers limit 5

Returned Researchers: 5
Time: 0.88s


id  count last_name   first_name  \
0   ur.01055753603.27    140    Hayase  Shuzi Shuzi   
1   ur.01144540527.52    103        Ma      Ting Li   
2  ur.011212042763.67    102    Hikita     Masayuki   
3  ur.016357156077.09     99        Lu       Huimin   
4   ur.07644453127.11     96    Kozako   M Kozako M   

                                       research_orgs               orcid_id  
0  [grid.14003.36, grid.419082.6, grid.266298.1, ...                    NaN  
1  [grid.258806.1, grid.177174.3, grid.411485.d, ...  [0000-0002-3310-459X]  
2      [grid.27476.30, grid.462727.2, grid.258806.1]                    NaN  
3  [grid.411497.e, grid.454850.8, grid.41156.37, ...  [0000-0001-9794-3221]  
4  [grid.471634.3, grid.482504.f, grid.258806.1, ...                    NaN

NOTE: facet results can be 1000 at most (due to performance limitations) so if there are more than 1000 it is not possible to know the total number. 

### 5.5 Paginating Results

At the end of a `return` phrase, the user can specify the maximum number
of results to be returned and the number of top records to skip over
before returning the first result record, for e.g. returning large
result sets page-by-page (i.e. "paging" results) as described below.

This is done using the keyword `limit` followed by the maximum number of
results to return, optionally followed by the keyword `skip` and the
number of results to skip (the offset).

In [55]:
%%dsldf 
search publications return publications limit 10

Returned Publications: 10 (total = 115232311)
Time: 1.18s


title     type              id  \
0  8 Gender Violence in the Hospitality Industry:...  chapter  pub.1134028727   
1  Mosaic Defects of AlN Buffer Layers in GaN/AlN...  article  pub.1126934999   
2  1 The Perils of Sex Work in Montreal: Seeking ...  chapter  pub.1134028720   
3  Gövde Buruşmalarının Elastik Olmayan Yanal Bur...  article  pub.1134154735   
4      Mağaza İmajının İkon Marka Algısına Yansıması  article  pub.1131456225   
5              Introduction: Accounting for Violence  chapter  pub.1134028719   
6  6 The Murder of Lori Dupont: Violence, Harassm...  chapter  pub.1134028725   
7  5 Slow Violence and Hidden Injuries: The Work ...  chapter  pub.1134028724   
8  4 Billy Gohl: Labour, Violence, and Myth in th...  chapter  pub.1134028723   
9  2 The Rules of Discipline: Workers and the Cul...  chapter  pub.1134028721   

     pages  year                                author_affiliations  \
0  184-198  2021                                                NaN   
1      1-1  2021  [[{'raw_affiliation': [], 'first_name': 'Tuğçe...   
2    16-41  2021                                                NaN   
3      1-1  2021  [[{'raw_affiliation': [], 'first_name': 'Mehme...   
4      NaN  2021  [[{'raw_affiliation': [], 'first_name': 'Yelda...   
5     1-15  2021                                                NaN   
6  133-159  2021                                                NaN   
7  112-132  2021                                                NaN   
8   88-111  2021                                                NaN   
9    42-61  2021                                                NaN   

     journal.id                           journal.title  
0           NaN                                     NaN  
1  jour.1152664                  Journal of Polytechnic  
2           NaN                                     NaN  
3  jour.1152664                  Journal of Polytechnic  
4  jour.1359387  Türkiye İletişim Araştırmaları Dergisi  
5           NaN                                     NaN  
6           NaN                                     NaN  
7           NaN                                     NaN  
8           NaN                                     NaN  
9           NaN                                     NaN

If paging information is not provided, the default values
`limit 20 skip 0` are used, so the two following queries are equivalent:

Combining `limit` and `skip` across multiple queries enables paging or
batching of results; e.g. to retrieve 30 grant records divided into 3
pages of 10 records each, the following three queries could be used:

```
return grants limit 10           => get 1st 10 records for page 1 (skip 0, by default)
return grants limit 10 skip 10   => get next 10 for page 2; skip the 10 we already have
return grants limit 10 skip 20   => get another 10 for page 3, for a total of 30
```

### 5.6 Sorting Results

A sort order for the results in a given `return` phrase can be specified
with the keyword `sort by` followed by the name of 
* a `field` (in the
case that a `source` is being requested) 
* an `indicator (aggregation)` (in the case
that one or more facets are being requested). 

 By default, the result set of full text
queries ($search ... for "full text query"$) is sorted by "relevance".
Additionally, it is possible to specify the sort order, using `asc` or
`desc` keywords. By default, descending order is selected.

In [56]:
%%dsldf 
search grants 
    for "nanomaterials"
return grants sort by title desc limit 5 

Returned Grants: 5 (total = 18755)
Time: 0.57s


title_language                                            funders  \
0             en  [{'id': 'grid.424150.6', 'acronym': 'DFG', 'co...   
1             en  [{'id': 'grid.424150.6', 'acronym': 'DFG', 'co...   
2             en  [{'id': 'grid.425119.a', 'acronym': 'BELSPO', ...   
3             pl  [{'id': 'grid.452947.9', 'acronym': 'FNP', 'co...   
4             pl  [{'id': 'grid.452947.9', 'acronym': 'FNP', 'co...   

                     original_title                       funding_org_name  \
0  Transmissionselektronenmikroskop             German Research Foundation   
1  Transmissionselektronenmikroskop             German Research Foundation   
2                      Snowcontrol.  Belgian Federal Science Policy Office   
3          Stypendium Naukowe START          Foundation for Polish Science   
4          Stypendium Naukowe START          Foundation for Polish Science   

       project_num             id                             title  \
0        280331443  grant.4841519  Transmissionselektronenmikroskop   
1        220923099  grant.4823271  Transmissionselektronenmikroskop   
2         3E120109  grant.6774902                      Snowcontrol.   
3    START 81.2014  grant.9182975                 START Scholarship   
4  START   79.2015  grant.9182996                 START Scholarship   

   start_date language                     active_year  start_year    end_date  
0  2015-01-01       en                          [2015]        2015         NaN  
1  2012-01-01       de                          [2012]        2012         NaN  
2  2011-06-16       en  [2011, 2012, 2013, 2014, 2015]        2011  2015-06-13  
3  2014-06-01       pl                    [2014, 2015]        2014  2015-06-01  
4  2015-06-01       pl                    [2015, 2016]        2015  2016-06-01

In [57]:
%%dsldf  
search grants  
    for "nanomaterials"
return grants  sort by relevance desc limit 5

Returned Grants: 5 (total = 18755)
Time: 0.57s


title_language                                            funders  \
0             en  [{'id': 'grid.437854.9', 'acronym': 'SFI', 'co...   
1             en  [{'id': 'grid.22919.31', 'acronym': 'FCT', 'co...   
2             en  [{'id': 'grid.22919.31', 'acronym': 'FCT', 'co...   
3             en  [{'id': 'grid.452912.9', 'acronym': 'NSERC', '...   
4             en  [{'id': 'grid.425339.a', 'acronym': 'ISF', 'co...   

                                      original_title  \
0              Optically-active chiral nanomaterials   
1  NOVEL LANTHANIDE LUMINESCENT SYSTEMS: FROM SUP...   
2  Transport properties and electrochemical appli...   
3                            catalytic nanomaterials   
4                   Novel biocomposite nanomaterials   

                                    funding_org_name   project_num  \
0                         Science Foundation Ireland  11/W.1/I2065   
1              Foundation for Science and Technology         35378   
2              Foundation for Science and Technology         39381   
3  Natural Sciences and Engineering Research Council        583037   
4                          Israel Science Foundation         25813   

              id                                              title  \
0  grant.3984032              Optically-active chiral nanomaterials   
1  grant.3526883  NOVEL LANTHANIDE LUMINESCENT SYSTEMS: FROM SUP...   
2  grant.3531153  Transport properties and electrochemical appli...   
3  grant.5527054                            catalytic nanomaterials   
4  grant.4849153                   Novel biocomposite nanomaterials   

   start_date    end_date language               active_year  start_year  
0  2012-06-01  2013-05-31       en              [2012, 2013]        2012  
1  2000-09-01  2003-12-31       en  [2000, 2001, 2002, 2003]        2000  
2  2003-03-01  2006-08-31       en  [2003, 2004, 2005, 2006]        2003  
3  2015-04-01  2016-03-31       en              [2015, 2016]        2015  
4  2012-01-01  2015-12-31       en  [2012, 2013, 2014, 2015]        2012

Number of citations per publication

In [58]:
%%dsldf  
search publications
return publications  [doi + times_cited] 
    sort by times_cited limit 5

Returned Publications: 5 (total = 115232310)
Time: 1.04s


times_cited                           doi
0       233883                           NaN
1       198871              10.1038/227680a0
2       183355  10.1016/0003-2697(76)90527-3
3        96100        10.1006/meth.2001.1262
4        89202   10.1103/physrevlett.77.3865

Recent citations per publication.
Note: Recent citation refers to the number of citations accrued in the last two year period. A single value is stored per document and the year window rolls over in July.

In [59]:
%%dsldf 
search publications
return publications [doi + recent_citations]
    sort by recent_citations limit 5

Returned Publications: 5 (total = 115232311)
Time: 1.31s


recent_citations                          doi
0             25499         10.1109/cvpr.2016.90
1             25295       10.1006/meth.2001.1262
2             22533           10.3322/caac.21492
3             19781  10.1103/physrevlett.77.3865
4             18649    10.1191/1478088706qp063oa

When a facet is being returned, the `indicator` used in the
`sort` phrase must either be `count` (the default, such that
`sort by count` is unnecessary), or one of the indicators specified in
the `aggregate` phrase, i.e. one whose values are being computed in the
faceting operation. 


In [60]:
%%dsldf 
search publications 
    for "nanomaterials"
return research_orgs 
    aggregate altmetric_median, rcr_avg sort by rcr_avg limit 5 

Returned Research_orgs: 5
Time: 2.62s


id  count     rcr_avg  altmetric_median    country_name  \
0  grid.11444.34      1  210.100006             350.0           China   
1  grid.11485.39      1  210.100006             350.0  United Kingdom   
2  grid.11642.30      1  210.100006             350.0         Reunion   
3  grid.20931.39      1  210.100006             350.0  United Kingdom   
4  grid.226688.0      1  210.100006             350.0       Singapore   

                                 name   latitude    city_name  \
0  Shanghai Institute of Hypertension  31.211678     Shanghai   
1                  Cancer Research UK  51.531322       London   
2            University of La Réunion -20.901735  Saint-Denis   
3            Royal Veterinary College  51.536800       London   
4    Temasek Life Sciences Laboratory   1.294417    Singapore   

                                             linkout   longitude        types  \
0                        [http://www.china-sih.com/]  121.467255   [Facility]   
1                 [http://www.cancerresearchuk.org/]   -0.106269  [Nonprofit]   
2  [http://www.univ-reunion.fr/university-of-reun...   55.484550  [Education]   
3                            [http://www.rvc.ac.uk/]   -0.134000  [Education]   
4                           [http://www.tll.org.sg/]  103.777000   [Facility]   

  acronym  
0     NaN  
1    CRUK  
2     NaN  
3     RVC  
4     TLL

### 5.7 Unnesting results

Multi-value entity and JSON fields, such as `researchers`, `authors` or `research_orgs` or any of `category_*` fields may be unnested into top level objects. 

This operation makes it easier to do further operations on these objects e.g. counting or processing them further. 

This functionality will transform all of the returned multi-value data and turn them into top level keys, such as `researchers.id`, `researchers.first_name`, `researchers.last_name`, while copying other, non-unnested fields, such as `id` or `title` of publication for each of them. Returned results are therefore multiplied by as many researchers and categories each original publication has, so they will likely be more than the overall query limit, as the limit applies on the source objects, not the unnested one. If multiple fields are being unnested, then a cartesian product of all unnested fields is being returned.




In [81]:
%%dsldf

search publications for "Japan AND Buddhism"
    where researchers is not empty
return publications[id+year+title+unnest(researchers)] limit 10

Returned objects: 12 (total publications= 39444)
Time: 1.60s


title  year              id  \
0   Tibetan CSL learners’ L2 Motivational Self Sys...  2021  pub.1133492334   
1   Connecting heritage, vulnerabilities and capac...  2021  pub.1133369377   
2   Robots are friends as well as foes: Ambivalent...  2021  pub.1132061246   
3   Robots are friends as well as foes: Ambivalent...  2021  pub.1132061246   
4   Informal institutions and comparative advantag...  2021  pub.1131260846   
5      Russian soft power from USSR to Putin’s Russia  2020  pub.1133271508   
6           Neoliberal capitalism and BRICS on screen  2020  pub.1133271504   
7                BRICS de-Americanizing the Internet?  2020  pub.1133271503   
8                              Contending soft powers  2020  pub.1133271506   
9                              Contending soft powers  2020  pub.1133271506   
10                                              India  2020  pub.1133224905   
11                                The BRICS paradox 1  2020  pub.1133224886   

        researchers.id researchers.first_name researchers.last_name  \
0   ur.014452013576.50                  Lubei                 Zhang   
1   ur.010763326731.79                 Ksenia              Chmutina   
2   ur.014753034505.67               Jianning                  Dang   
3    ur.01036243756.64                   Ling                   Liu   
4   ur.011016512337.06                 Pao-Li                 Chang   
5   ur.011543771702.89                Elena V                Bykova   
6   ur.015265002403.21                  Iiris                 Ruoho   
7   ur.015745210637.94            Daya Kishan                Thussu   
8   ur.010352300011.45                 Herman             Wasserman   
9   ur.010227271267.80            Musawenkosi                Ndlovu   
10  ur.015745210637.94            Daya Kishan                Thussu   
11  ur.016411703546.16                  Jyrki               Käkönen   

                            researchers.research_orgs   researchers.orcid_id  
0                                     [grid.263901.f]                    NaN  
1                          [grid.4563.4, grid.6571.5]                    NaN  
2       [grid.27860.3b, grid.412498.2, grid.20513.35]  [0000-0002-8174-0136]  
3   [grid.418213.d, grid.16753.36, grid.11135.37, ...  [0000-0002-4898-3013]  
4                      [grid.214458.e, grid.412634.6]  [0000-0001-5485-7374]  
5                                     [grid.15447.33]                    NaN  
6                                     [grid.502801.e]                    NaN  
7   [grid.12896.34, grid.23231.31, grid.8096.7, gr...                    NaN  
8                                                 NaN                    NaN  
9                                     [grid.442325.6]  [0000-0002-5901-6766]  
10  [grid.12896.34, grid.23231.31, grid.8096.7, gr...                    NaN  
11                       [grid.502801.e, grid.8207.d]                    NaN

In [78]:
%%dsldf

search publications for "Japan AND Buddhism"
return publications[id+year+title+unnest(category_for)] limit 5

Returned objects: 10 (total publications= 120925)
Time: 0.90s


year                                              title              id  \
0  2021  Tibetan CSL learners’ L2 Motivational Self Sys...  pub.1133492334   
1  2021  Tibetan CSL learners’ L2 Motivational Self Sys...  pub.1133492334   
2  2021  A creative destruction approach to replication...  pub.1133082954   
3  2021  A creative destruction approach to replication...  pub.1133082954   
4  2021  The shaping of anticipation: The networked dev...  pub.1134273988   
5  2021  The shaping of anticipation: The networked dev...  pub.1134273988   
6  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
7  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
8  2021  Learning in retirement: Developing resilience ...  pub.1131496119   
9  2021  Learning in retirement: Developing resilience ...  pub.1131496119   

  category_for.id                     category_for.name  
0            3268  1303 Specialist Studies In Education  
1            2213                          13 Education  
2            2217  17 Psychology and Cognitive Sciences  
3            3468                       1701 Psychology  
4            3448                        1608 Sociology  
5            2216           16 Studies in Human Society  
6            2217  17 Psychology and Cognitive Sciences  
7            3468                       1701 Psychology  
8            2217  17 Psychology and Cognitive Sciences  
9            3468                       1701 Psychology

You can `unnest` as many fields as you want. However the number of results will grow pretty quickly!

In [82]:
%%dsldf

search publications for "Japan AND Buddhism"
return publications[id+year+title+unnest(category_for)+unnest(researchers)+unnest(research_orgs)] limit 5

Returned objects: 42 (total publications= 120925)
Time: 0.99s


year                                              title              id  \
0   2021  Tibetan CSL learners’ L2 Motivational Self Sys...  pub.1133492334   
1   2021  Tibetan CSL learners’ L2 Motivational Self Sys...  pub.1133492334   
2   2021  A creative destruction approach to replication...  pub.1133082954   
3   2021  A creative destruction approach to replication...  pub.1133082954   
4   2021  A creative destruction approach to replication...  pub.1133082954   
5   2021  A creative destruction approach to replication...  pub.1133082954   
6   2021  A creative destruction approach to replication...  pub.1133082954   
7   2021  A creative destruction approach to replication...  pub.1133082954   
8   2021  A creative destruction approach to replication...  pub.1133082954   
9   2021  A creative destruction approach to replication...  pub.1133082954   
10  2021  A creative destruction approach to replication...  pub.1133082954   
11  2021  A creative destruction approach to replication...  pub.1133082954   
12  2021  A creative destruction approach to replication...  pub.1133082954   
13  2021  A creative destruction approach to replication...  pub.1133082954   
14  2021  A creative destruction approach to replication...  pub.1133082954   
15  2021  A creative destruction approach to replication...  pub.1133082954   
16  2021  A creative destruction approach to replication...  pub.1133082954   
17  2021  A creative destruction approach to replication...  pub.1133082954   
18  2021  A creative destruction approach to replication...  pub.1133082954   
19  2021  A creative destruction approach to replication...  pub.1133082954   
20  2021  A creative destruction approach to replication...  pub.1133082954   
21  2021  A creative destruction approach to replication...  pub.1133082954   
22  2021  A creative destruction approach to replication...  pub.1133082954   
23  2021  A creative destruction approach to replication...  pub.1133082954   
24  2021  A creative destruction approach to replication...  pub.1133082954   
25  2021  A creative destruction approach to replication...  pub.1133082954   
26  2021  A creative destruction approach to replication...  pub.1133082954   
27  2021  A creative destruction approach to replication...  pub.1133082954   
28  2021  A creative destruction approach to replication...  pub.1133082954   
29  2021  A creative destruction approach to replication...  pub.1133082954   
30  2021  A creative destruction approach to replication...  pub.1133082954   
31  2021  A creative destruction approach to replication...  pub.1133082954   
32  2021  The shaping of anticipation: The networked dev...  pub.1134273988   
33  2021  The shaping of anticipation: The networked dev...  pub.1134273988   
34  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
35  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
36  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
37  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
38  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
39  2021  Exploring the gratitude model of body apprecia...  pub.1134199615   
40  2021  Learning in retirement: Developing resilience ...  pub.1131496119   
41  2021  Learning in retirement: Developing resilience ...  pub.1131496119   

   category_for.id                     category_for.name research_orgs.id  \
0             3268  1303 Specialist Studies In Education    grid.263901.f   
1             2213                          13 Education    grid.263901.f   
2             2217  17 Psychology and Cognitive Sciences    grid.273335.3   
3             2217  17 Psychology and Cognitive Sciences    grid.262273.0   
4             2217  17 Psychology and Cognitive Sciences    grid.148374.d   
5             2217  17 Psychology and Cognitive Sciences    grid.17088.36   
6             2217  17 Psychology and Cognitive Sciences    grid.27

## 6. Aggregations

In a `return` phrase requesting one or more `facet` results, aggregation
operations to perform during faceting can be specified after the facet
name(s) by using the keyword `aggregate` followed by a comma-separated
list of one or more `indicator` names corresponding to the `source`
being searched.

In [61]:
%%dsldf
search publications 
    where year > 2010 
return research_orgs  
    aggregate rcr_avg, altmetric_median limit 5

Returned Research_orgs: 5
Time: 13.43s


id   count   rcr_avg  altmetric_median    country_name  \
0  grid.38142.3c  198744  2.193411               5.0   United States   
1  grid.17063.33  150759  1.732056               4.0          Canada   
2  grid.26999.3d  149047  1.205105               2.0           Japan   
3  grid.11899.38  144382  1.077967               2.0          Brazil   
4  grid.83440.3b  131707  1.962088               4.0  United Kingdom   

                        name   latitude  city_name  \
0         Harvard University  42.377052  Cambridge   
1      University of Toronto  43.661667    Toronto   
2        University of Tokyo  35.713333      Tokyo   
3    University of São Paulo -23.563051  São Paulo   
4  University College London  51.524470     London   

                          linkout   longitude     state_name        types  \
0       [http://www.harvard.edu/]  -71.116650  Massachusetts  [Education]   
1       [http://www.utoronto.ca/]  -79.395000        Ontario  [Education]   
2  [http://www.u-tokyo.ac.jp/en/]  139.762220            NaN  [Education]   
3        [http://www5.usp.br/en/]  -46.730103            NaN  [Education]   
4         [http://www.ucl.ac.uk/]   -0.133982            NaN  [Education]   

  acronym  
0     NaN  
1     NaN  
2      UT  
3     USP  
4     UCL

**What are the metrics/aggregations available?** See the data sources documentation for information about available [indicators](https://docs.dimensions.ai/dsl/datasource-publications.html#publications-indicators).  

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [62]:
schema = dsl.query("describe schema")
sources = [x for x in schema['sources']]
# for each source name, extract metrics info
for s in sources:
    print("SOURCE:", s)
    for m in schema['sources'][s]['metrics']:
        print("--", schema['sources'][s]['metrics'][m]['name'], " => ", schema['sources'][s]['metrics'][m]['description'], )

SOURCE: publications
-- count  =>  Total count
-- altmetric_median  =>  Median Altmetric attention score
-- altmetric_avg  =>  Altmetric attention score mean
-- citations_total  =>  Aggregated number of citations
-- citations_avg  =>  Arithmetic mean of citations
-- citations_median  =>  Median of citations
-- recent_citations_total  =>  For a given article, in a given year, the number of citations accrued in the last two year period. Single value stored per document, year window rolls over in July.
-- rcr_avg  =>  Arithmetic mean of `relative_citation_ratio` field.
-- fcr_gavg  =>  Geometric mean of `field_citation_ratio` field (note: This field cannot be used for sorting results).
SOURCE: grants
-- count  =>  Total count
-- funding  =>  Total funding amount, in USD.
SOURCE: patents
-- count  =>  Total count
SOURCE: clinical_trials
-- count  =>  Total count
SOURCE: policy_documents
-- count  =>  Total count
SOURCE: researchers
-- count  =>  Total count
SOURCE: organizations
-- count  

**NOTE** In addition to any specified aggregations, `count` is always computed
and reported when facet results are requested.

In [63]:
%%dsldf
search grants 
    for "5g network" 
return funders 
    aggregate count, funding sort by funding limit 5 

Returned Funders: 5
Time: 0.57s


id  count      funding    country_name   acronym   longitude  \
0  grid.270680.b    202  969574372.0         Belgium        EC    4.363670   
1  grid.457785.c    141   70847172.0   United States  NSF CISE  -77.111000   
2  grid.421091.f     68   53295321.0  United Kingdom     EPSRC   -1.784602   
3  grid.55047.33      8   50521157.0          Poland      NCRD   21.007630   
4  grid.453115.7     40   35972874.0           China       ITC  114.166580   

          types  city_name                                      linkout  \
0  [Government]   Brussels           [http://ec.europa.eu/index_en.htm]   
1  [Government]  Arlington  [http://www.nsf.gov/dir/index.jsp?org=CISE]   
2  [Government]    Swindon                   [https://www.epsrc.ac.uk/]   
3  [Government]     Warsaw                 [http://www.ncbr.gov.pl/en/]   
4  [Government]  Hong Kong     [http://www.itc.gov.hk/en/about/org.htm]   

                                                name   latitude state_name  
0                                European Commission  50.851650        NaN  
1  Directorate for Computer & Information Science...  38.880580   Virginia  
2  Engineering and Physical Sciences Research Cou...  51.567093    England  
3       National Centre for Research and Development  52.227455        NaN  
4               Innovation and Technology Commission  22.282640        NaN

Aggregated total number of citations

In [64]:
%%dsldf
search publications
    for "ontologies"
return funders 
    aggregate citations_total 
    sort by citations_total  limit 5

Returned Funders: 5
Time: 0.91s


id  count  citations_total   country_name  \
0  grid.280785.0  13704         895803.0  United States   
1  grid.48336.3a  13440         892092.0  United States   
2  grid.270680.b  20684         650861.0        Belgium   
3  grid.280128.1   5040         648335.0  United States   
4  grid.419696.5  36530         495653.0          China   

                                             name   latitude  city_name  \
0  National Institute of General Medical Sciences  38.997833   Bethesda   
1                       National Cancer Institute  39.004326  Rockville   
2                             European Commission  50.851650   Brussels   
3        National Human Genome Research Institute  38.996967   Bethesda   
4    National Natural Science Foundation of China  40.005177    Beijing   

                                         linkout  longitude acronym  \
0  [http://www.nigms.nih.gov/Pages/default.aspx]  -77.09938   NIGMS   
1                       [http://www.cancer.gov/]  -77.10119     NCI   
2             [http://ec.europa.eu/index_en.htm]    4.36367      EC   
3                      [https://www.genome.gov/]  -77.09693   NHGRI   
4      [http://www.nsfc.gov.cn/publish/portal1/]  116.33983    NSFC   

  state_name         types  
0   Maryland    [Facility]  
1   Maryland  [Government]  
2        NaN  [Government]  
3   Maryland    [Facility]  
4        NaN  [Government]

Arithmetic mean number of citations

In [65]:
%%dsldf
search publications
return funders 
    aggregate citations_avg 
    sort by citations_avg limit 5

Returned Funders: 5
Time: 1.96s


id  count  citations_avg    country_name            state_name  \
0  grid.478308.0    190     264.768421   United States  District of Columbia   
1  grid.417710.4    154     215.993506   United States              Maryland   
2  grid.453780.d    145     193.931034   United States  District of Columbia   
3  grid.484432.d      1     188.000000  United Kingdom                   NaN   
4  grid.478789.d    576     171.751736   United States                Nevada   

    longitude        types        city_name                          linkout  \
0  -77.039730  [Nonprofit]  Washington D.C.  [http://www.stewart-trust.org/]   
1  -77.203760    [Company]        Rockville            [http://www.hgsi.com]   
2  -77.039520  [Nonprofit]  Washington D.C.           [http://www.abc2.org/]   
3   -0.123164  [Nonprofit]           London  [https://www.macmillan.org.uk/]   
4 -115.299850      [Other]        Las Vegas     [http://www.dwreynolds.org/]   

                                    name   latitude                   acronym  
0     Alexander & Margaret Stewart Trust  38.901160                       NaN  
1  Human Genome Sciences (United States)  39.096650                       NaN  
2           Accelerate Brain Cancer Cure  38.906720                       NaN  
3               Macmillan Cancer Support  51.488003  Macmillan Cancer Support  
4          Donald W. Reynolds Foundation  36.190460                       NaN

Geometric mean of FCR


In [66]:
%%dsldf
search publications
return funders 
    aggregate fcr_gavg limit 5

Returned Funders: 5
Time: 3.41s


id  fcr_gavg    count   country_name  \
0  grid.419696.5  2.364073  2162769          China   
1  grid.270680.b  3.289544   763648        Belgium   
2  grid.424020.0  2.580812   668039          China   
3  grid.54432.34  2.279734   619365          Japan   
4  grid.48336.3a  4.913860   588545  United States   

                                                name   latitude  city_name  \
0       National Natural Science Foundation of China  40.005177    Beijing   
1                                European Commission  50.851650   Brussels   
2  Ministry of Science and Technology of the Peop...  39.827835    Beijing   
3         Japan Society for the Promotion of Science  35.687160      Tokyo   
4                          National Cancer Institute  39.004326  Rockville   

                                     linkout   longitude acronym  \
0  [http://www.nsfc.gov.cn/publish/portal1/]  116.339830    NSFC   
1         [http://ec.europa.eu/index_en.htm]    4.363670      EC   
2              [http://www.most.gov.cn/eng/]  116.316284    MOST   
3                   [http://www.jsps.go.jp/]  139.740390    JSPS   
4                   [http://www.cancer.gov/]  -77.101190     NCI   

          types state_name  
0  [Government]        NaN  
1  [Government]        NaN  
2  [Government]        NaN  
3   [Nonprofit]        NaN  
4  [Government]   Maryland

Median Altmetric Attention Score

In [67]:
%%dsldf 
search publications
return funders aggregate altmetric_median 
    sort by altmetric_median limit 5 

Returned Funders: 5
Time: 6.64s


id  count  altmetric_median    country_name  \
0  grid.470711.4      2             117.0  United Kingdom   
1  grid.443873.f      5              98.0   United States   
2  grid.473856.b      2              66.0   United States   
3  grid.481336.b      1              44.0  United Kingdom   
4  grid.419979.b      2              43.5   United States   

                                       name   latitude        city_name  \
0           Chest Heart and Stroke Scotland  55.946075        Edinburgh   
1                      LUNGevity Foundation  41.878674          Chicago   
2  Administration for Children and Families  38.885940  Washington D.C.   
3                     Target Ovarian Cancer  51.529922           London   
4               Einstein Healthcare Network  40.036827     Philadelphia   

                                    linkout  longitude                acronym  \
0                 [http://www.chss.org.uk/]  -3.219597                   CHSS   
1               [http://www.lungevity.org/] -87.626480                   LUNG   
2                [https://www.acf.hhs.gov/] -77.016370                    ACF   
3  [http://www.targetovariancancer.org.uk/]  -0.101692  Target Ovarian Cancer   
4                [http://www.einstein.edu/] -75.143140                   AEHN   

          types            state_name  
0   [Nonprofit]                   NaN  
1   [Nonprofit]              Illinois  
2  [Government]  District of Columbia  
3   [Nonprofit]                   NaN  
4  [Healthcare]          Pennsylvania

### 6.1 Complex aggregations

The `return` phrase may be followed by a function expression, to return additional calculations, such as per year funding or citations statistics. These functions may take their own arguments, and are calculated using the source data as specified in the `search part` of the query.

At the time of writing, there are two functions available: Publications `citations_per_year` and Grants `funding_per_year`

#### Publications `citations_per_year`

Publication citations is the number of times that publications have been cited by other publications in the database. This function returns the number of citations received in each year.

In [73]:
%%dsldf

search publications for "brexit"
return citations_per_year(2010, 2020)

Returned Citations_per_year: 11
Time: 3.91s


citations_per_year
2010                 6.0
2011                11.0
2012                10.0
2013                14.0
2014                25.0
2015               107.0
2016               737.0
2017              5382.0
2018             16401.0
2019             32265.0
2020             54327.0

#### Grants `funding_per_year`

Returns grant funding per year in the given currency, starting from specified year, ending in specified year (including).

Supported currencies are: CAD,USD,JPY,GBP,CHF,CNY,EUR,NZD,AUD

In [74]:
%%dsldf

search grants for "brexit"
return funding_per_year(2010, 2020, "USD")


Returned Funding_per_year: 11
Time: 0.60s


funding_per_year
2010               0.0
2011               0.0
2012               0.0
2013            4412.0
2014           10020.0
2015          342762.0
2016          823304.0
2017         5973847.0
2018        15563940.0
2019        34619271.0
2020        42622951.0